access to the data_storage.py: https://drive.google.com/file/d/1-6_x0L6_yxaj3oxwmGJoYbn6luBgcnwX/view?usp=drive_link
access to the code below as script: https://drive.google.com/file/d/1-9158gNZZzkJLlUvEiqUkq6S7cVNLMf4/view?usp=sharing

In [1]:
import ee
# @title Authenticate to the Earth Engine servers
ee.Authenticate()
# Initialize the Earth Engine object with Google Cloud project ID
project_id = 'ee-janet' # change here
ee.Initialize(project=project_id)


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data')

Mounted at /content/drive


In [13]:
# @title Lib imports:
#import ee
#print('Using EE version ', ee.__version__)
import folium
#print('Using Folium version ', folium.__version__)
from os import MFD_HUGE_1MB
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Iterable, List, Tuple
#from google.colab import auth
import datetime as dt
import time
import geopandas as gpd
from shapely.geometry import shape, Polygon, MultiPolygon
import scripts.data_storage as ds  # Importing the data storage script
import scripts.extract_s2_planet_ncfi as esp #import extract s2_planet_ncfi

ModuleNotFoundError: No module named 'scripts.extract_s2_planet_ncfi'

## Clean the raw data for all years

In [7]:
import os
os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/clean_data_no_bands')

In [9]:
shp_2018 = gpd.read_file('land_cover_2018.shp')#.apply(lambda x: x.astype(str).apply(capitalize_first))
shp_2019 = gpd.read_file('land_cover_2019.shp')#.apply(lambda x: x.astype(str).apply(capitalize_first))
shp_2020 = gpd.read_file('land_cover_2020.shp')#.apply(lambda x: x.astype(str).apply(capitalize_first))
shp_2020 = shp_2020[shp_2020['ID'].notna()]
shp_2023 = gpd.read_file('land_cover_2023.shp')#.apply(lambda x: x.astype(str).apply(capitalize_first))

In [10]:
shp_2023.head()

,ID,Class,Name,Sub_class,Year,geometry
0,1,Crops,Cashew,Tree_Crops,2023,"POLYGON ((-16.35692 13.85091, -16.35663 13.851..."
1,2,Crops,Cashew,Tree_Crops,2023,"POLYGON ((-16.37232 13.82824, -16.37223 13.828..."
2,3,Crops,Cashew,Tree_Crops,2023,"POLYGON ((-16.42729 13.75103, -16.42726 13.751..."
3,4,Crops,Cashew,Tree_Crops,2023,"POLYGON ((-16.42793 13.76519, -16.42789 13.765..."
4,5,Crops,Cashew,Tree_Crops,2023,"POLYGON ((-16.42801 13.76906, -16.42797 13.769..."


In [11]:
shp_2020.Name.unique()

array(['Millet_Cowpea', 'Millet_Bissap', 'Groundnut_Mixed', 'Millet',
       'Cowpea', 'Sorghum', 'Fallow', 'Maize', 'Fonio', 'Laterite_Road',
       'Groundnut', 'Vegetation', 'Millet_Pasture', 'Built_Up',
       'Paved_Road', 'Bissap', 'Cowpea_Bissap', 'Watermelon',
       'Sandy_Road', 'Maize_Other', 'Maize_Millet', 'Maize_Bissap',
       'Rice', 'Cotton', 'Track', 'Maize_Gum', 'Okra', 'Sesame',
       'Bare_Soil', 'Quarry', 'School', 'Cowpea_Pasture',
       'Cowpea_Soybean', 'Cemetery', 'Sesame_Bissap', 'Sesame_Millet',
       'Okra_Maize', 'Sugarcane', 'Bissap_Groundnut',
       'Cotton_Other_Crops', 'Millet_Maize', 'Maize_Sorghum',
       'Millet_Groundnut'], dtype=object)

### Extracting remote sensing indices and normalized band values

In [ ]:
class Sentinel2Processor:
    def __init__(self, year, months, indices, data_fc):
        self.year = year
        self.months = months
        self.indices = indices
        self.data_fc = data_fc

    def calculate_indices(self, image):
        index_functions = {
            'NDVI': lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')
        }

        for index in self.indices:
            if index in index_functions:
                image = image.addBands(index_functions[index](image))

        return image

    def normalize_band(self, image, band_name):
        band = image.select(band_name).toFloat()
        min_max = band.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=image.geometry(),
            scale=10,
            maxPixels=1e13
        )
        min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
        max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))
        normalized = band.subtract(min_val).divide(max_val.subtract(min_val)).rename(ee.String(band_name))#.cat('_n'))
        return normalized

    def process_image(self, image):
        with_indices = self.calculate_indices(image)
        bands_to_normalize = ['B2', 'B3', 'B4', 'B5', 'B8', 'B11', 'B12']
        normalized_bands = ee.Image.cat([self.normalize_band(with_indices, band) for band in bands_to_normalize])
        return normalized_bands.addBands(with_indices.select(self.indices))

    def process_all_months(self):
        processed_images = []

        for month_index, month in enumerate(self.months):
            start_date = ee.Date.fromYMD(self.year, month, 1)
            end_date = ee.Date.fromYMD(self.year, ee.Number(month).add(1), 1)

            collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
                .filterBounds(self.data_fc) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

            num_images = self.retry_function(lambda: collection.size().getInfo())
            print(f"Number of images for {month}/{self.year}: {num_images}")

            processed_collection = collection.map(self.process_image)
            composite = processed_collection.median().toFloat()
            # Create the date string in the format "MMYY"
            date_str = f'{month:02d}{str(self.year)[-2:]}'

            def rename_band(band_name):
                return ee.String('norm_').cat(band_name).cat('_').cat(date_str)

            new_band_names = composite.bandNames().map(rename_band)
            print(new_band_names.getInfo())
            composite = composite.rename(new_band_names)

            processed_images.append(composite)

            # Add a delay between processing months
            time.sleep(random.uniform(5, 10))

        return ee.ImageCollection(processed_images).toBands()

    def retry_function(self, func, max_retries=10, initial_delay=1, factor=2):
        """Retry a function with exponential backoff."""
        retries = 0
        while retries < max_retries:
            try:
                return func()
            except ee.ee_exception.EEException as e:
                if "Too many concurrent aggregations" in str(e):
                    retries += 1
                    if retries == max_retries:
                        raise
                    delay = initial_delay * (factor ** retries) + random.uniform(0, 1)
                    print(f"Rate limit hit. Retrying in {delay:.2f} seconds...")
                    time.sleep(delay)
                else:
                    raise

    def add_bands_to_fc(self, image):
        def sample_image(feature):
            values = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=10,
                maxPixels=1e13
            )
            return feature.set(values)

        return self.data_fc.map(sample_image)

    def export_to_drive(self, description, file_format='SHP', folder='clean_data_with_bands_s2'):
        combined_image = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(combined_image)
        print(data_with_bands.getInfo())

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(Sentinel2Processor):
    def __init__(self, subclass_df, year, months, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, months, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = []
            for polygon in geometry.geoms:
                polygons.append(list(polygon.exterior.coords))
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    import os

    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/clean_data_with_bands_s2')

    # Years to process
    years = [2018]#, 2019, 2020, 2023]  # Replace with your desired years
    months = [8]#, 9, 10, 11]  # Add or reduce months
    indices = ['NDVI']  # Add more indices as you wish

    shp_data = {
        2018: shp_2018,
      #  2019: shp_2019,
       # 2020: shp_2020,
      #  2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            #subclass_df= subclass_df.apply(lambda x: x.astype(str).apply(capitalize_first))
            processor = SubclassProcessor(subclass_df, year, months, indices, subclass_name)
            try:
                processor.export_to_drive(f'data_{year}_with_bands_subclass_{subclass_name}')
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(10, 20))


In [ ]:
# Main execution
if __name__ == "__main__":
    import os
    import ee

    # Initialize Earth Engine
    ee.Initialize()

    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/clean_data_with_bands_s2')

    # Years to process
    years = [2018, 2019, 2020, 2023]
    months = [8, 9, 10, 11]  # Add or reduce months as needed
    indices = ['NDVI']  # Add more indices as you wish

    # Create a dictionary to store the FeatureCollections for each year and subclass
    data_fc = {
        2018: {
            'Cereals': clean_raw_data_2018_Cereals,
            'Legumes': clean_raw_data_2018_Legumes,
            'Noncrop': clean_raw_data_2018_Noncrop,
            'Tree_Crops': clean_raw_data_2018_Tree_Crops,
            'Vegetables': clean_raw_data_2018_Vegetables
        },
        2019: {
            'Cereals': clean_raw_data_2019_Cereals,
            'Legumes': clean_raw_data_2019_Legumes,
            'Noncrop': clean_raw_data_2019_Noncrop,
            'Vegetables': clean_raw_data_2019_Vegetables
        },
        2020: {
            'Bare_Built_Up': clean_raw_data_2020_Bare_Built_Up,
            'Cereals': clean_raw_data_2020_Cereals,
            'Fallow': clean_raw_data_2020_Fallow,
            'Legumes': clean_raw_data_2020_Legumes,
            'Noncrop': clean_raw_data_2020_Noncrop,
            'Other_Vegetation': clean_raw_data_2020_Other_Vegetation,
            'Tree_Crops': clean_raw_data_2020_Tree_Crops,
            'Vegetables': clean_raw_data_2020_Vegetables
        },
        2023: {
            'Cereals': clean_raw_data_2023_Cereals,
            'Fallow': clean_raw_data_2023_Fallow,
            'Legumes': clean_raw_data_2023_Legumes,
            'Noncrop': clean_raw_data_2023_Noncrop,
            'Tree_Crops': clean_raw_data_2023_Tree_Crops,
            'Vegetables': clean_raw_data_2023_Vegetables
        }
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, fc in data_fc[year].items():
            processor = esp.SubclassProcessor(year, months, indices, fc)
            try:
                processor.export_to_drive(f'data_{year}_with_bands_subclass_{subclass_name}')
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(10, 20))

In [ ]:

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/clean_data_with_bands_s2')

    # Years to process
    years = [2018,2019, 2020, 2023]  #2019, 2020, 2023 # Replace with your desired years
    months = [8, 9, 10, 11]  # add or reduce months
    indices = ['NDVI']  # add more indices as you wish

    # Assuming 'shp_data' is a dictionary with years as keys and GeoDataFrames as values
    # You need to define these GeoDataFrames (shp_2018, shp_2019, shp_2020, shp_2023) before this point
    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, months, indices, subclass_name)
            try:
                processor.export_to_drive(f'data_{year}_with_bands_subclass_{subclass_name}')
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(5, 10))

Planet NICFI

$`Biannual Collection`
   PS_Tropical_Normalized_Analytic_Biannual

*  December 2015,  June 2016, December 2016, June 2017, December 2017,   June 2018,December 2018, June 2019, December 2019, June 2020
                              
                          

$`Monthly Collection`
  PS_Tropical_Normalized_Analytic_Monthly
* September 2020, October 2020,November 2020, December 2020,January 2021,February 2021, March 2021,April 2021,May 2021,June 2021,July 2021,August 2021, September 2021,October 2021,November 2021,December 2021,January 2022, February 2022,March 2022,  April 2022
                       

In [ ]:
# Set working directory
os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data')

# Load shapefiles
shp_2018 = gpd.read_file('clean_data_no_bands/land_cover_2018.shp')
shp_2019 = gpd.read_file('clean_data_no_bands/land_cover_2019.shp')
shp_2020 = gpd.read_file('clean_data_no_bands/land_cover_2020.shp')
shp_2023 = gpd.read_file('clean_data_no_bands/land_cover_2023.shp')

class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges  # List of tuples (start_date, end_date)
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        """
        Calculate the NDVI for an image and add it as a band.
        """
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        """
        Normalize the specified bands of an image.
        """
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=4.77,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))

            # Check if min and max are valid
            is_valid = min_val.isNumber().And(max_val.isNumber()).And(max_val.neq(min_val))

            normalized_band = ee.Image(ee.Algorithms.If(
                is_valid,
                band.subtract(min_val).divide(max_val.subtract(min_val)),
                band  # If normalization fails, return the original band
            )).rename(ee.String(band_name).cat('_norm'))

            return normalized_band

        normalized_bands = [normalize_band(band) for band in band_names]
        return image.addBands(ee.Image.cat(normalized_bands))

    def process_all_months(self):
        """
        Process the NICFI Planet imagery for all specified date ranges.
        """
        processed_images = []

        for start_date, end_date in self.date_ranges:
            print(f"Processing period: {start_date} to {end_date}")

            nicfi_planet = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa") \
                .filterBounds(self.data_fc.geometry()) \
                .filter(ee.Filter.date(start_date, end_date))

            # Check if the collection is empty
            count = nicfi_planet.size().getInfo()
            if count == 0:
                print(f"No images found for the period: {start_date} to {end_date}")
                continue

            normalized_collection = nicfi_planet.map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry()))
            ndvi_collection = normalized_collection.map(self.calculate_ndvi)

            print(f"Number of images: {ndvi_collection.size().getInfo()}")

            bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

            composite = ndvi_collection.median().select(bands)

            # Add a date band to the composite
            date_band = ee.Image.constant(ee.Date(start_date).millis()).rename('date')
            composite_with_date = composite.addBands(date_band)

            processed_images.append(composite_with_date)

            time.sleep(random.uniform(5, 10))

        if not processed_images:
            raise ValueError("No images were processed for any date range")

        return ee.ImageCollection(processed_images)

    def add_bands_to_fc(self, image_collection):
        """
        Add the calculated bands to the feature collection as time series.
        """
        def sample_image_collection(feature):
            values = image_collection.map(lambda image: image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=4.77,
                maxPixels=1e13
            )).toList(image_collection.size())

            return feature.set('time_series', values)

        return self.data_fc.map(sample_image_collection)

    def export_to_drive(self, description, file_format='CSV', folder='clean_data_with_bands_planet_ncifi'):
        """
        Export the feature collection with added bands to Google Drive.
        """
        image_collection = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(image_collection)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    """
    Convert a GeoDataFrame to an Earth Engine Feature Collection.
    """
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = [list(polygon.exterior.coords) for polygon in geometry.geoms]
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-06-01', '2018-08-31'),
            ('2018-06-01', '2018-09-30'),
            ('2018-06-01', '2018-10-31'),
            ('2018-06-01', '2018-11-30')
        ],
        2019: [
            ('2019-06-01', '2019-08-31'),
            ('2019-06-01', '2019-09-30'),
            ('2019-06-01', '2019-10-31'),
            ('2019-06-01', '2019-11-30')
        ],
        2020: [
            ('2020-08-01', '2020-08-31'),
            ('2020-09-01', '2020-09-30'),
            ('2020-10-01', '2020-10-31'),
            ('2020-11-01', '2020-11-30')
        ],
        2023: [
            ('2023-08-01', '2023-08-31'),
            ('2023-09-01', '2023-09-30'),
            ('2023-10-01', '2023-10-31'),
            ('2023-11-01', '2023-11-30')
        ]
    }

    indices = ['NDVI']  # Add more indices as you wish

    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")
            except ValueError as e:
                print(f"No data available for {year}, subclass {subclass_name}: {str(e)}")
            except Exception as e:
                print(f"Unexpected error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(10, 20))

In [ ]:

class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges  # List of tuples (start_date, end_date)
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        """
        Calculate the NDVI for an image and add it as a band.
        """
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        """
        Normalize the specified bands of an image.
        """
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=4.77,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))

            # Check if min and max are valid
            is_valid = min_val.lt(max_val)

            normalized_band = ee.Image(ee.Algorithms.If(
                is_valid,
                band.subtract(min_val).divide(max_val.subtract(min_val)),
                band  # If normalization fails, return the original band
            )).rename(ee.String(band_name).cat('_norm'))

            return normalized_band

        normalized_bands = ee.ImageCollection(ee.List(band_names).map(lambda band: normalize_band(ee.String(band))))
        normalized_image = normalized_bands.toBands().rename(band_names)

        return image.addBands(normalized_image)

    def process_all_months(self):
        """
        Process the NICFI Planet imagery for all specified date ranges.
        """
        processed_images = []

        for start_date, end_date in self.date_ranges:
            print(f"Processing period: {start_date} to {end_date}")

            nicfi_planet = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa") \
                .filterBounds(self.data_fc.geometry()) \
                .filter(ee.Filter.date(start_date, end_date))

            # Check if the collection is empty
            count = nicfi_planet.size()

            def process_collection(count):
                normalized_collection = nicfi_planet.map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry()))
                ndvi_collection = normalized_collection.map(self.calculate_ndvi)

                print(f"Number of images: {count.getInfo()}")

                composite = ndvi_collection.median()

                # Add a date band to the composite
                date_band = ee.Image.constant(ee.Date(start_date).millis()).rename('date')
                return composite.addBands(date_band)

            composite_with_date = ee.Algorithms.If(
                count.gt(0),
                process_collection(count),
                None
            )

            processed_images.append(composite_with_date)

            time.sleep(random.uniform(5, 10))

        # Filter out None values
        processed_images = [img for img in processed_images if img is not None]

        if not processed_images:
            raise ValueError("No images were processed for any date range")

        return ee.ImageCollection(processed_images)

    def add_bands_to_fc(self, image_collection):
        """
        Add the calculated bands to the feature collection as time series.
        """
        def sample_image_collection(feature):
            values = image_collection.map(lambda image: image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=4.77,
                maxPixels=1e13
            )).toList(image_collection.size())

            return feature.set('time_series', values)

        return self.data_fc.map(sample_image_collection)

    def export_to_drive(self, description, file_format='CSV', folder='clean_data_with_bands_planet_ncifi'):
        """
        Export the feature collection with added bands to Google Drive.
        """
        image_collection = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(image_collection)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    """
    Convert a GeoDataFrame to an Earth Engine Feature Collection.
    """
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = [list(polygon.exterior.coords) for polygon in geometry.geoms]
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    import geopandas as gpd

    # Set up your Google Drive folder
    drive_folder = '/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi'
    if not os.path.exists(drive_folder):
        os.makedirs(drive_folder)
    os.chdir(drive_folder)

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-06-01', '2018-08-31'),
            ('2018-06-01', '2018-09-30'),
            ('2018-06-01', '2018-10-31'),
            ('2018-06-01', '2018-11-30')
        ],
        2019: [
            ('2019-06-01', '2019-08-31'),
            ('2019-06-01', '2019-09-30'),
            ('2019-06-01', '2019-10-31'),
            ('2019-06-01', '2019-11-30')
        ],
        2020: [
            ('2020-08-01', '2020-08-31'),
            ('2020-09-01', '2020-09-30'),
            ('2020-10-01', '2020-10-31'),
            ('2020-11-01', '2020-11-30')
        ],
        2023: [
            ('2023-08-01', '2023-08-31'),
            ('2023-09-01', '2023-09-30'),
            ('2023-10-01', '2023-10-31'),
            ('2023-11-01', '2023-11-30')
        ]
    }

    indices = ['NDVI']  # Add more indices as you wish



    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")
            except ValueError as e:
                print(f"No data available for {year}, subclass {subclass_name}: {str(e)}")
            except Exception as e:
                print(f"Unexpected error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(10, 20))


Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: 6IOPORXOQY5H52ZUZU3KYNMJ
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: POTMARXPEOQZ4L6OPRQSWPYH
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: VNFFL

In [ ]:
class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges  # List of tuples (start_date, end_date)
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        """
        Calculate the NDVI for an image and add it as a band.
        """
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        """
        Normalize the specified bands of an image.
        """
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=4.77,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))

            # Check if min and max are valid
            is_valid = min_val.lt(max_val)

            normalized_band = ee.Image(ee.Algorithms.If(
                is_valid,
                band.subtract(min_val).divide(max_val.subtract(min_val)),
                band  # If normalization fails, return the original band
            )).rename(ee.String(band_name).cat('_norm'))

            return normalized_band

        normalized_bands = ee.ImageCollection(ee.List(band_names).map(lambda band: normalize_band(ee.String(band))))
        normalized_image = normalized_bands.toBands().rename(band_names)

        return image.addBands(normalized_image)

    def process_all_months(self):
        """
        Process the NICFI Planet imagery for all specified date ranges.
        """
        processed_images = []

        for start_date, end_date in self.date_ranges:
            print(f"Processing period: {start_date} to {end_date}")

            nicfi_planet = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa") \
                .filterBounds(self.data_fc.geometry()) \
                .filter(ee.Filter.date(start_date, end_date))

            # Check if the collection is empty
            count = nicfi_planet.size()

            def process_collection(count):
                normalized_collection = nicfi_planet.map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry()))
                ndvi_collection = normalized_collection.map(self.calculate_ndvi)

                print(f"Number of images: {count.getInfo()}")

                composite = ndvi_collection.median()

                # Add a date band to the composite
                date_band = ee.Image.constant(ee.Date(start_date).millis()).rename('date')
                return composite.addBands(date_band)

            composite_with_date = ee.Algorithms.If(
                count.gt(0),
                process_collection(count),
                None
            )

            processed_images.append(composite_with_date)

            time.sleep(random.uniform(5, 10))

        # Filter out None values
        processed_images = [img for img in processed_images if img is not None]

        if not processed_images:
            raise ValueError("No images were processed for any date range")

        return ee.ImageCollection(processed_images)

    def add_bands_to_fc(self, image_collection):
        """
        Add the calculated bands to the feature collection as time series.
        """
        def sample_image_collection(feature):
            values = image_collection.map(lambda image: image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=4.77,
                maxPixels=1e13
            )).toList(image_collection.size())

            return feature.set('time_series', values)

        return self.data_fc.map(sample_image_collection)

    def export_to_drive(self, description, file_format='CSV', folder='clean_data_with_bands_planet_ncifi'):
        """
        Export the feature collection with added bands to Google Drive.
        """
        image_collection = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(image_collection)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    """
    Convert a GeoDataFrame to an Earth Engine Feature Collection.
    """
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = [list(polygon.exterior.coords) for polygon in geometry.geoms]
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-06-01', '2018-08-31'),
            ('2018-06-01', '2018-09-30'),
            ('2018-06-01', '2018-10-31'),
            ('2018-06-01', '2018-11-30')
        ],
        2019: [
            ('2019-06-01', '2019-08-31'),
            ('2019-06-01', '2019-09-30'),
            ('2019-06-01', '2019-10-31'),
            ('2019-06-01', '2019-11-30')
        ],
        2020: [
            ('2020-08-01', '2020-08-31'),
            ('2020-09-01', '2020-09-30'),
            ('2020-10-01', '2020-10-31'),
            ('2020-11-01', '2020-11-30')
        ],
        2023: [
            ('2023-08-01', '2023-08-31'),
            ('2023-09-01', '2023-09-30'),
            ('2023-10-01', '2023-10-31'),
            ('2023-11-01', '2023-11-30')
        ]
    }

    indices = ['NDVI']  # Add more indices as you wish

    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")
            except ValueError as e:
                print(f"No data available for {year}, subclass {subclass_name}: {str(e)}")
            except Exception as e:
                print(f"Unexpected error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(10, 20))

Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: K55JY52IN7H5X3IR46AQR2QZ
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: U75JJWWRAY567E6GNHZBXGRT
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: KGEKD

KeyboardInterrupt: 

In [ ]:
# Set working directory
os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data')

# Load shapefiles
shp_2018 = gpd.read_file('clean_data_no_bands/land_cover_2018.shp')
shp_2019 = gpd.read_file('clean_data_no_bands/land_cover_2019.shp')
shp_2020 = gpd.read_file('clean_data_no_bands/land_cover_2020.shp')
shp_2023 = gpd.read_file('clean_data_no_bands/land_cover_2023.shp')

class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges  # List of tuples (start_date, end_date)
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        """
        Calculate the NDVI for an image and add it as a band.
        """
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        """
        Normalize the specified bands of an image.
        """
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=4.77,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))

            # Check if min and max are valid
            is_valid = min_val.lt(max_val)

            normalized_band = ee.Image(ee.Algorithms.If(
                is_valid,
                band.subtract(min_val).divide(max_val.subtract(min_val)),
                band  # If normalization fails, return the original band
            )).rename(ee.String(band_name).cat('_norm'))

            return normalized_band

        normalized_bands = ee.List(band_names).map(lambda band: normalize_band(ee.String(band)))
        return image.addBands(normalized_bands)

    def process_all_months(self):
        """
        Process the NICFI Planet imagery for all specified date ranges.
        """
        processed_images = []

        for start_date, end_date in self.date_ranges:
            print(f"Processing period: {start_date} to {end_date}")

            nicfi_planet = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa") \
                .filterBounds(self.data_fc.geometry()) \
                .filter(ee.Filter.date(start_date, end_date))

            # Check if the collection is empty
            count = nicfi_planet.size()

            def process_collection(count):
                normalized_collection = nicfi_planet.map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry()))
                ndvi_collection = normalized_collection.map(self.calculate_ndvi)

                print(f"Number of images: {count.getInfo()}")

                composite = ndvi_collection.mosaic()

                # Add a date band to the composite
                date_band = ee.Image.constant(ee.Date(start_date).millis()).rename('date')
                return composite.addBands(date_band)

            composite_with_date = ee.Algorithms.If(
                count.gt(0),
                process_collection(count),
                None
            )

            processed_images.append(composite_with_date)

            time.sleep(random.uniform(5, 10))

        # Filter out None values
        processed_images = [img for img in processed_images if img is not None]

        if not processed_images:
            raise ValueError("No images were processed for any date range")

        return ee.ImageCollection(processed_images)

    def add_bands_to_fc(self, image_collection):
        """
        Add the calculated bands to the feature collection as time series.
        """
        def sample_image_collection(feature):
            values = image_collection.map(lambda image: image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=4.77,
                maxPixels=1e13
            )).toList(image_collection.size())

            return feature.set('time_series', values)

        return self.data_fc.map(sample_image_collection)

    def export_to_drive(self, description, file_format='CSV', folder='clean_data_with_bands_planet_ncifi'):
        """
        Export the feature collection with added bands to Google Drive.
        """
        image_collection = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(image_collection)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    """
    Convert a GeoDataFrame to an Earth Engine Feature Collection.
    """
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = [list(polygon.exterior.coords) for polygon in geometry.geoms]
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-06-01', '2018-08-31'),
            ('2018-06-01', '2018-09-30'),
            ('2018-06-01', '2018-10-31'),
            ('2018-06-01', '2018-11-30')
        ],
        2019: [
            ('2019-06-01', '2019-08-31'),
            ('2019-06-01', '2019-09-30'),
            ('2019-06-01', '2019-10-31'),
            ('2019-06-01', '2019-11-30')
        ],
        2020: [
            ('2020-08-01', '2020-08-31'),
            ('2020-09-01', '2020-09-30'),
            ('2020-10-01', '2020-10-31'),
            ('2020-11-01', '2020-11-30')
        ],
        2023: [
            ('2023-08-01', '2023-08-31'),
            ('2023-09-01', '2023-09-30'),
            ('2023-10-01', '2023-10-31'),
            ('2023-11-01', '2023-11-30')
        ]
    }

    indices = ['NDVI']  # Add more indices as you wish

    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")
            except ValueError as e:
                print(f"No data available for {year}, subclass {subclass_name}: {str(e)}")
            except Exception as e:
                print(f"Unexpected error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(10, 20))

Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: BBG6TPJSXAKURVKLAEKCCB6B
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: ZJLSFN6JU7ZVVF2AGK5OKSJT
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: OIHRI

KeyboardInterrupt: 

In [ ]:
# Set working directory
os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data')

# Load shapefiles
shp_2018 = gpd.read_file('clean_data_no_bands/land_cover_2018.shp')
shp_2019 = gpd.read_file('clean_data_no_bands/land_cover_2019.shp')
shp_2020 = gpd.read_file('clean_data_no_bands/land_cover_2020.shp')
shp_2023 = gpd.read_file('clean_data_no_bands/land_cover_2023.shp')

class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges  # List of tuples (start_date, end_date)
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        """
        Calculate the NDVI for an image and add it as a band.
        """
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        """
        Normalize the specified bands of an image.
        """
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=4.77,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))

            # Check if min and max are valid
            is_valid = min_val.lt(max_val)

            normalized_band = ee.Image(ee.Algorithms.If(
                is_valid,
                band.subtract(min_val).divide(max_val.subtract(min_val)),
                band  # If normalization fails, return the original band
            )).rename(ee.String(band_name).cat('_norm'))

            return normalized_band

        normalized_bands = ee.List(band_names).map(lambda band: normalize_band(ee.String(band)))
        return image.addBands(ee.Image.cat(normalized_bands))

    def process_all_months(self):
        """
        Process the NICFI Planet imagery for all specified date ranges.
        """
        processed_images = []

        for start_date, end_date in self.date_ranges:
            print(f"Processing period: {start_date} to {end_date}")

            nicfi_planet = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa") \
                .filterBounds(self.data_fc.geometry()) \
                .filter(ee.Filter.date(start_date, end_date))

            # Check if the collection is empty
            count = nicfi_planet.size()

            def process_collection(count):
                normalized_collection = nicfi_planet.map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry()))
                ndvi_collection = normalized_collection.map(self.calculate_ndvi)

                print(f"Number of images: {count.getInfo()}")

                bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

                composite = ndvi_collection.median().select(bands)

                # Add a date band to the composite
                date_band = ee.Image.constant(ee.Date(start_date).millis()).rename('date')
                return composite.addBands(date_band)

            composite_with_date = ee.Algorithms.If(
                count.gt(0),
                process_collection(count),
                None
            )

            processed_images.append(composite_with_date)

            time.sleep(random.uniform(5, 10))

        # Filter out None values
        processed_images = [img for img in processed_images if img is not None]

        if not processed_images:
            raise ValueError("No images were processed for any date range")

        return ee.ImageCollection(processed_images)

    def add_bands_to_fc(self, image_collection):
        """
        Add the calculated bands to the feature collection as time series.
        """
        def sample_image_collection(feature):
            values = image_collection.map(lambda image: image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=4.77,
                maxPixels=1e13
            )).toList(image_collection.size())

            return feature.set('time_series', values)

        return self.data_fc.map(sample_image_collection)

    def export_to_drive(self, description, file_format='CSV', folder='clean_data_with_bands_planet_ncifi'):
        """
        Export the feature collection with added bands to Google Drive.
        """
        image_collection = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(image_collection)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    """
    Convert a GeoDataFrame to an Earth Engine Feature Collection.
    """
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = [list(polygon.exterior.coords) for polygon in geometry.geoms]
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-06-01', '2018-08-31'),
            ('2018-06-01', '2018-09-30'),
            ('2018-06-01', '2018-10-31'),
            ('2018-06-01', '2018-11-30')
        ],
        2019: [
            ('2019-06-01', '2019-08-31'),
            ('2019-06-01', '2019-09-30'),
            ('2019-06-01', '2019-10-31'),
            ('2019-06-01', '2019-11-30')
        ],
        2020: [
            ('2020-08-01', '2020-08-31'),
            ('2020-09-01', '2020-09-30'),
            ('2020-10-01', '2020-10-31'),
            ('2020-11-01', '2020-11-30')
        ],
        2023: [
            ('2023-08-01', '2023-08-31'),
            ('2023-09-01', '2023-09-30'),
            ('2023-10-01', '2023-10-31'),
            ('2023-11-01', '2023-11-30')
        ]
    }

    indices = ['NDVI']  # Add more indices as you wish

    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")
            except ValueError as e:
                print(f"No data available for {year}, subclass {subclass_name}: {str(e)}")
            except Exception as e:
                print(f"Unexpected error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(10, 20))

Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: YGEAOLK4YNRFBZGX6WNH5MKH
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: 4EIX54B34JG7EQKQ4XBMVLJG
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: 2YMVJ

KeyboardInterrupt: 

In [ ]:
# Set working directory
os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data')

# Load shapefiles
shp_2018 = gpd.read_file('clean_data_no_bands/land_cover_2018.shp')
shp_2019 = gpd.read_file('clean_data_no_bands/land_cover_2019.shp')
shp_2020 = gpd.read_file('clean_data_no_bands/land_cover_2020.shp')
shp_2023 = gpd.read_file('clean_data_no_bands/land_cover_2023.shp')

class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges  # List of tuples (start_date, end_date)
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        """
        Calculate the NDVI for an image and add it as a band.
        """
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        """
        Normalize the specified bands of an image.
        """
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=4.77,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))
            normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).rename(ee.String(band_name).cat('_norm'))
            return normalized_band

        normalized_bands = [normalize_band(band) for band in band_names]
        return image.addBands(ee.Image.cat(normalized_bands))

    def process_all_months(self):
        """
        Process the NICFI Planet imagery for all specified date ranges.
        """
        processed_images = []

        for start_date, end_date in self.date_ranges:
            print(f"Processing period: {start_date} to {end_date}")

            nicfi_planet = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa") \
                .filterBounds(self.data_fc.geometry()) \
                .filter(ee.Filter.date(start_date, end_date)) \
                .map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry())) \
                .map(self.calculate_ndvi)

            print(f"Number of images: {nicfi_planet.size().getInfo()}")

            bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

            composite = nicfi_planet.median().select(bands)
           # print(composite.getInfo())

            # Add a date band to the composite
            date_band = ee.Image.constant(ee.Date(start_date).millis()).rename('date')
            composite_with_date = composite.addBands(date_band)

            processed_images.append(composite_with_date)

            time.sleep(random.uniform(5, 10))

        return ee.ImageCollection(processed_images)

    def add_bands_to_fc(self, image_collection):
        """
        Add the calculated bands to the feature collection as time series.
        """
        def sample_image_collection(feature):
            values = image_collection.map(lambda image: image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=4.77,
                maxPixels=1e13
            )).toList(image_collection.size())

            return feature.set('time_series', values)

        return self.data_fc.map(sample_image_collection)

    def export_to_drive(self, description, file_format='CSV', folder='clean_data_with_bands_planet_ncifi'):
        """
        Export the feature collection with added bands to Google Drive.
        """
        image_collection = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(image_collection)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    """
    Convert a GeoDataFrame to an Earth Engine Feature Collection.
    """
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = [list(polygon.exterior.coords) for polygon in geometry.geoms]
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [ 2018, 2019,2020, 2023]#[

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-06-01', '2018-08-31'),
            ('2018-06-01', '2018-09-30'),
            ('2018-06-01', '2018-10-31'),
            ('2018-06-01', '2018-11-30')
        ],
        2019: [
            ('2019-06-01', '2019-08-31'),
            ('2019-06-01', '2019-09-30'),
            ('2019-06-01', '2019-10-31'),
            ('2019-06-01', '2019-11-30')
        ],
        2020: [
            ('2020-08-01', '2020-08-31'),
            ('2020-09-01', '2020-09-30'),
            ('2020-10-01', '2020-10-31'),
            ('2020-11-01', '2020-11-30')
        ],
        2023: [
            ('2023-08-01', '2023-08-31'),
            ('2023-09-01', '2023-09-30'),
            ('2023-10-01', '2023-10-31'),
            ('2023-11-01', '2023-11-30')
        ]
    }

    indices = ['NDVI']  # Add more indices as you wish

    shp_data = {
        2018: shp_2018,
       2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(10,20))


Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: ZTXZZPKI6SYLKQEXFI7UMMLG
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: 6RX65HSTQSKIIUU4RRP5PTRK
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-06-01 to 2018-08-31
Number of images: 1
Processing period: 2018-06-01 to 2018-09-30
Number of images: 1
Processing period: 2018-06-01 to 2018-10-31
Number of images: 1
Processing period: 2018-06-01 to 2018-11-30
Number of images: 1
Started export task: NDS3K

KeyboardInterrupt: 

In [ ]:
class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges  # List of tuples (start_date, end_date)
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        """
        Calculate the NDVI for an image and add it as a band.
        """
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        """
        Normalize the specified bands of an image.
        """
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=4.77,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))
            normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).rename(ee.String(band_name).cat('_norm'))
            return normalized_band

        normalized_bands = [normalize_band(band) for band in band_names]
        return image.addBands(ee.Image.cat(normalized_bands))

    def process_all_months(self):
        """
        Process the NICFI Planet imagery for all specified date ranges.
        """
        processed_images = []

        for start_date, end_date in self.date_ranges:
            print(f"Processing period: {start_date} to {end_date}")

            nicfi_planet =ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa")\
                .filterBounds(self.data_fc.geometry()) \
                .filter(ee.Filter.date(start_date, end_date)) \
                .map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry())) \
                .map(self.calculate_ndvi)

            print(f"Number of images: {nicfi_planet.getInfo()}")

            bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

            composite = nicfi_planet.median().select(bands)

            # Add a date band to the composite
            date_band = ee.Image.constant(ee.Date(start_date).millis()).rename('date')
            composite_with_date = composite.addBands(date_band)

            processed_images.append(composite_with_date)

            time.sleep(random.uniform(1, 3))

        return ee.ImageCollection(processed_images)

    def add_bands_to_fc(self, image_collection):
        """
        Add the calculated bands to the feature collection as time series.
        """
        def sample_image_collection(feature):
            values = image_collection.map(lambda image: image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=5,
                maxPixels=1e13
            )).toList(image_collection.size())

            return feature.set('time_series', values)

        return self.data_fc.map(sample_image_collection)

    def export_to_drive(self, description, file_format='CSV', folder='clean_data_with_bands'):
        """
        Export the feature collection with added bands to Google Drive.
        """
        image_collection = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(image_collection)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    """
    Convert a GeoDataFrame to an Earth Engine Feature Collection.
    """
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = [list(polygon.exterior.coords) for polygon in geometry.geoms]
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-08-01', '2018-08-31'),
            # ('2018-09-01', '2018-09-30'),
            # ('2018-10-01', '2018-10-31'),
            # ('2018-11-01', '2018-11-30')
        ],
        # 2019: [
        #     ('2019-08-01', '2019-08-31'),
        #     ('2019-09-01', '2019-09-30'),
        #     ('2019-10-01', '2019-10-31'),
        #     ('2019-11-01', '2019-11-30')
        # ],
        # 2020: [
        #     ('2020-08-01', '2020-08-31'),
        #     ('2020-09-01', '2020-09-30'),
        #     ('2020-10-01', '2020-10-31'),
        #     ('2020-11-01', '2020-11-30')
        # ],
        # 2023: [
        #     ('2023-08-01', '2023-08-31'),
        #     ('2023-09-01', '2023-09-30'),
        #     ('2023-10-01', '2023-10-31'),
        #     ('2023-11-01', '2023-11-30')
        # ]
    }

    indices = ['NDVI']  # Add more indices as you wish

    # Assuming 'shp_data' is a dictionary with years as keys and GeoDataFrames as values
    # You need to define these GeoDataFrames (shp_2018, shp_2019, shp_2020, shp_2023) before this point
    shp_data = {
        2018: shp_2018,
       # 2019: shp_2019,
       # 2020: shp_2020,
       # 2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(5, 10))

Processing period: 2018-08-01 to 2018-08-31
Number of images: {'type': 'ImageCollection', 'bands': [], 'version': 1721366263582886, 'id': 'projects/planet-nicfi/assets/basemaps/africa', 'properties': {'area': 'Tropical Africa ', 'system:time_start': 1598918400000, 'spectral_resolution': 'R,G,B,NIR', 'system:time_end': 1601424000000, 'description': 'This image collection provides access to high-resolution satellite monitoring of the tropics for the primary purpose of reducing and reversing the loss of tropical forests, contributing to combating climate change, conserving biodiversity, contributing to forest regrowth, restoration and enhancement, and facilitating sustainable development, all of which must be Non-Commercial Use. To gain access you must sign the agreement available here: https://www.planet.com/nicfi/\n\n\nIn support of NICFI’s mission, you can use this data for a number of projects including, but not limited to:\n\n\n\xa0- Advance scientific research about the world’s trop

KeyboardInterrupt: 

In [ ]:

class PlanetNICFIProcessor:
    def __init__(self, year, months, indices, data_fc):
        self.year = year
        self.months = months
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        """
        Calculate the NDVI for an image and add it as a band.
        """
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        """
        Normalize the specified bands of an image.
        """
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=5,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))
            normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).rename(ee.String(band_name).cat('_norm'))
            #print(f"Normalized {band_name} with min: {min_val.getInfo()}, max: {max_val.getInfo()}")
            return normalized_band

        normalized_bands = [normalize_band(band) for band in band_names]
        return image.addBands(ee.Image.cat(normalized_bands))

    def process_all_months(self):
        """
        Process the NICFI Planet imagery for all specified months.
        """

        for month_index, month in enumerate(self.months):
            start_date = ee.Date.fromYMD(self.year, month, 1)
            end_date = ee.Date.fromYMD(self.year, ee.Number(month).add(1), 1)
            print(start_date.getInfo(), end_date.getInfo())
            # Load NICFI/Planet imagery and filter by date and bounds of the polygons
            nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
                .filterBounds(self.data_fc.geometry()) \
                .filterDate(start_date, end_date)\
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry())) \
                .map(self.calculate_ndvi)

            # Select bands to use for classification
            bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

            # Reduce the image collection to median to get a single composite image
            composite = nicfi_planet.median().select(bands)
           #processed_images.append(composite)

            # Add a delay between processing months
            time.sleep(random.uniform(1, 3))
        return composite


    def retry_function(self, func, max_retries=5, initial_delay=1, factor=2):
        """
        Retry a function with exponential backoff in case of rate limit errors.
        """
        retries = 0
        while retries < max_retries:
            try:
                return func()
            except ee.ee_exception.EEException as e:
                if "Too many concurrent aggregations" in str(e):
                    retries += 1
                    if retries == max_retries:
                        raise
                    delay = initial_delay * (factor ** retries) + random.uniform(0, 1)
                    print(f"Rate limit hit. Retrying in {delay:.2f} seconds...")
                    time.sleep(delay)
                else:
                    raise

    def add_bands_to_fc(self, image):
        """
        Add the calculated bands to the feature collection.
        """
        def sample_image(feature):
            values = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=5,
                maxPixels=1e13
            )
            return feature.set(values)

        return self.data_fc.map(sample_image)

    def export_to_drive(self, description, file_format='SHP', folder='clean_data_with_bands'):
        """
        Export the feature collection with added bands to Google Drive.
        """
        combined_image = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(combined_image)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, months, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, months, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    """
    Convert a GeoDataFrame to an Earth Engine Feature Collection.
    """
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = [list(polygon.exterior.coords) for polygon in geometry.geoms]
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)


In [ ]:
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-08-01', '2018-08-31'),
            ('2018-09-01', '2018-09-30'),
            ('2018-10-01', '2018-10-31'),
            ('2018-11-01', '2018-11-30')
        ],
        2019: [
            ('2019-08-01', '2019-08-31'),
            ('2019-09-01', '2019-09-30'),
            ('2019-10-01', '2019-10-31'),
            ('2019-11-01', '2019-11-30')
        ],
        # Add ranges for 2020 and 2023
    }

    indices = ['NDVI']  # Add more indices as you wish

    # Assuming 'shp_data' is a dictionary with years as keys and GeoDataFrames as values
    # You need to define these GeoDataFrames (shp_2018, shp_2019, shp_2020, shp_2023) before this point
    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        #2020: shp_2020,
        #2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                processor.export_to_drive(f'data_{year}_with_bands_subclass_{subclass_name}')
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(5, 10))

TypeError: not all arguments converted during string formatting

In [ ]:
class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges  # List of tuples (start_date, end_date)
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        """
        Calculate the NDVI for an image and add it as a band.
        """
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        """
        Normalize the specified bands of an image.
        """
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=5,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))
            normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).rename(ee.String(band_name).cat('_norm'))
            return normalized_band

        normalized_bands = [normalize_band(band) for band in band_names]
        return image.addBands(ee.Image.cat(normalized_bands))

    def process_all_months(self):
        """
        Process the NICFI Planet imagery for all specified date ranges.
        """
        processed_images = []

        for start_date, end_date in self.date_ranges:
            print(f"Processing period: {start_date} to {end_date}")

            nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
                .filterBounds(self.data_fc.geometry()) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry())) \
                .map(self.calculate_ndvi)

            bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

            composite = nicfi_planet.median().select(bands)
            processed_images.append(composite)

            time.sleep(random.uniform(1, 3))

        return ee.Image.cat(processed_images)

    def retry_function(self, func, max_retries=5, initial_delay=1, factor=2):
        """
        Retry a function with exponential backoff in case of rate limit errors.
        """
        retries = 0
        while retries < max_retries:
            try:
                return func()
            except ee.ee_exception.EEException as e:
                if "Too many concurrent aggregations" in str(e):
                    retries += 1
                    if retries == max_retries:
                        raise
                    delay = initial_delay * (factor ** retries) + random.uniform(0, 1)
                    print(f"Rate limit hit. Retrying in {delay:.2f} seconds...")
                    time.sleep(delay)
                else:
                    raise

    def add_bands_to_fc(self, image):
        """
        Add the calculated bands to the feature collection.
        """
        def sample_image(feature):
            values = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=5,
                maxPixels=1e13
            )
            return feature.set(values)

        return self.data_fc.map(sample_image)

    def export_to_drive(self, description, file_format='SHP', folder='clean_data_with_bands'):
        """
        Export the feature collection with added bands to Google Drive.
        """
        combined_image = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(combined_image)
        print(data_with_bands.getInfo())

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    """
    Convert a GeoDataFrame to an Earth Engine Feature Collection.
    """
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = [list(polygon.exterior.coords) for polygon in geometry.geoms]
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-08-01', '2018-08-31'),
            ('2018-09-01', '2018-09-30'),
            ('2018-10-01', '2018-10-31'),
            ('2018-11-01', '2018-11-30')
         ],
        # 2019: [
        #     ('2019-08-01', '2019-08-31'),
        #     ('2019-09-01', '2019-09-30'),
        #     ('2019-10-01', '2019-10-31'),
        #     ('2019-11-01', '2019-11-30')
        # ],
        # 2020: [
        #     ('2020-08-01', '2020-08-31'),
        #     ('2020-09-01', '2020-09-30'),
        #     ('2020-10-01', '2020-10-31'),
        #     ('2020-11-01', '2020-11-30')
        # ],
        # 2023: [
        #     ('2023-08-01', '2023-08-31'),
        #     ('2023-09-01', '2023-09-30'),
        #     ('2023-10-01', '2023-10-31'),
        #     ('2023-11-01', '2023-11-30')
        # ]
    }

    indices = ['NDVI']  # Add more indices as you wish

    # Assuming 'shp_data' is a dictionary with years as keys and GeoDataFrames as values
    # You need to define these GeoDataFrames (shp_2018, shp_2019, shp_2020, shp_2023) before this point
    shp_data = {
        2018: shp_2018,
        #2019: shp_2019,
       # 2020: shp_2020,
        #2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(5, 10))

Processing period: 2018-08-01 to 2018-08-31
Processing period: 2018-09-01 to 2018-09-30
Processing period: 2018-10-01 to 2018-10-31
Processing period: 2018-11-01 to 2018-11-30
Error processing 2018, subclass cereals: Image.select: Band pattern 'B_norm' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
Processing period: 2018-08-01 to 2018-08-31
Processing period: 2018-09-01 to 2018-09-30
Processing period: 2018-10-01 to 2018-10-31
Processing period: 2018-11-01 to 2018-11-30
Error processing 2018, subclass legumes: Image.select: Band pattern 'B_norm' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
Processing period: 2018-08-01 to 2018-08-31
Processing period: 2018-09-01 to 2018-09-30
Processing period: 2018-10-01 to 2018-10-31
Processing period: 2018-11-01 to 2018-11-30
Error processing 2018, subclass noncrop: Image.select: Band pattern 'B_norm' was applied to an

KeyError: 2019

In [ ]:
class PlanetNICFIProcessor:
    def __init__(self, year, months, indices, data_fc):
        self.year = year
        self.months = months
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=5,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))
            return band.subtract(min_val).divide(max_val.subtract(min_val)).rename(ee.String(band_name).cat('_norm'))

        normalized_bands = [normalize_band(band) for band in band_names]
        return image.addBands(ee.Image.cat(normalized_bands))

    def process_all_months(self):

      processed_images = []

      for start_date, end_date in self.date_ranges:
          print(f"Processing period: {start_date} to {end_date}")

          nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
              .filterBounds(self.data_fc.geometry()) \
              .filterDate(start_date, end_date) \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

          # Check if the collection is empty
          count = nicfi_planet.size().getInfo()
          if count == 0:
              print(f"No images found for the period {start_date} to {end_date}")
              continue

          # Print the bands of the first image
          first_image = ee.Image(nicfi_planet.first())
          print(f"Bands in the original image: {first_image.bandNames().getInfo()}")

          normalized_collection = nicfi_planet.map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry()))

          # Check the bands after normalization
          first_normalized = ee.Image(normalized_collection.first())
          print(f"Bands after normalization: {first_normalized.bandNames().getInfo()}")

          with_ndvi = normalized_collection.map(self.calculate_ndvi)

          bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

          composite = with_ndvi.median()

          # Check if all required bands are present
          available_bands = composite.bandNames().getInfo()
          missing_bands = [band for band in bands if band not in available_bands]
          if missing_bands:
              print(f"Warning: The following bands are missing: {missing_bands}")
              # Only select available bands
              bands = [band for band in bands if band in available_bands]

          if not bands:
              print("Error: No required bands are available in the composite image")
              continue

          composite = composite.select(bands)
          processed_images.append(composite)

          time.sleep(random.uniform(1, 3))

      if not processed_images:
          raise ValueError("No images were processed successfully")

      return ee.Image.cat(processed_images)

    def retry_function(self, func, max_retries=5, initial_delay=1, factor=2):
        """Retry a function with exponential backoff."""
        retries = 0
        while retries < max_retries:
            try:
                return func()
            except ee.ee_exception.EEException as e:
                if "Too many concurrent aggregations" in str(e):
                    retries += 1
                    if retries == max_retries:
                        raise
                    delay = initial_delay * (factor ** retries) + random.uniform(0, 1)
                    print(f"Rate limit hit. Retrying in {delay:.2f} seconds...")
                    time.sleep(delay)
                else:
                    raise

    def add_bands_to_fc(self, image):
        def sample_image(feature):
            values = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=5,
                maxPixels=1e13
            )
            return feature.set(values)

        return self.data_fc.map(sample_image)

    def export_to_drive(self, description, file_format='SHP', folder='data_with_bands_planet_ncifi'):
        combined_image = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(combined_image)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = []
            for polygon in geometry.geoms:
                polygons.append(list(polygon.exterior.coords))
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-08-01', '2018-08-31'),
            ('2018-09-01', '2018-09-30'),
            ('2018-10-01', '2018-10-31'),
            ('2018-11-01', '2018-11-30')
        ],
        2019: [
            ('2019-08-01', '2019-08-31'),
            ('2019-09-01', '2019-09-30'),
            ('2019-10-01', '2019-10-31'),
            ('2019-11-01', '2019-11-30')
        ],
        2020: [
            ('2020-08-01', '2020-08-31'),
            ('2020-09-01', '2020-09-30'),
            ('2020-10-01', '2020-10-31'),
            ('2020-11-01', '2020-11-30')
        ],
        2023: [
            ('2023-08-01', '2023-08-31'),
            ('2023-09-01', '2023-09-30'),
            ('2023-10-01', '2023-10-31'),
            ('2023-11-01', '2023-11-30')
        ]
    }

    indices = ['NDVI']  # Add more indices as you wish

    # Assuming 'shp_data' is a dictionary with years as keys and GeoDataFrames as values
    # You need to define these GeoDataFrames (shp_2018, shp_2019, shp_2020, shp_2023) before this point
    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(5, 10))

AttributeError: 'SubclassProcessor' object has no attribute 'date_ranges'

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 19 23:39:24 2024

@author: Janet.Mutuku

This script processes land cover data for various years using Google Earth Engine (GEE) and outputs the results as shapefiles.
"""

import ee
import time
import random
import os
from shapely.geometry import Polygon, MultiPolygon

# Initialize Earth Engine
ee.Initialize()

class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=5,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))
            return band.subtract(min_val).divide(max_val.subtract(min_val)).rename(ee.String(band_name).cat('_norm'))

        normalized_bands = [normalize_band(band) for band in band_names]
        return image.addBands(ee.Image.cat(normalized_bands))

    def process_all_months(self):
        processed_images = []

        for start_date, end_date in self.date_ranges:
            print(f"Processing period: {start_date} to {end_date}")

            nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
                .filterBounds(self.data_fc.geometry()) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

            # Check if the collection is empty
            count = nicfi_planet.size().getInfo()
            if count == 0:
                print(f"No images found for the period {start_date} to {end_date}")
                continue

            # Print the bands of the first image
            first_image = ee.Image(nicfi_planet.first())
            print(f"Bands in the original image: {first_image.bandNames().getInfo()}")

            normalized_collection = nicfi_planet.map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry()))

            # Check the bands after normalization
            first_normalized = ee.Image(normalized_collection.first())
            print(f"Bands after normalization: {first_normalized.bandNames().getInfo()}")

            with_ndvi = normalized_collection.map(self.calculate_ndvi)

            bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

            composite = with_ndvi.median()

            # Check if all required bands are present
            available_bands = composite.bandNames().getInfo()
            missing_bands = [band for band in bands if band not in available_bands]
            if missing_bands:
                print(f"Warning: The following bands are missing: {missing_bands}")
                # Only select available bands
                bands = [band for band in bands if band in available_bands]

            if not bands:
                print("Error: No required bands are available in the composite image")
                continue

            composite = composite.select(bands)
            processed_images.append(composite)

            time.sleep(random.uniform(1, 3))

        if not processed_images:
            raise ValueError("No images were processed successfully")

        return ee.Image.cat(processed_images)

    def retry_function(self, func, max_retries=5, initial_delay=1, factor=2):
        """Retry a function with exponential backoff."""
        retries = 0
        while retries < max_retries:
            try:
                return func()
            except ee.ee_exception.EEException as e:
                if "Too many concurrent aggregations" in str(e):
                    retries += 1
                    if retries == max_retries:
                        raise
                    delay = initial_delay * (factor ** retries) + random.uniform(0, 1)
                    print(f"Rate limit hit. Retrying in {delay:.2f} seconds...")
                    time.sleep(delay)
                else:
                    raise

    def add_bands_to_fc(self, image):
        def sample_image(feature):
            values = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=5,
                maxPixels=1e13
            )
            return feature.set(values)

        return self.data_fc.map(sample_image)

    def export_to_drive(self, description, file_format='SHP', folder='data_with_bands_planet_ncifi'):
        combined_image = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(combined_image)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = []
            for polygon in geometry.geoms:
                polygons.append(list(polygon.exterior.coords))
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year
    date_ranges = {
        2018: [
            ('2018-08-01', '2018-08-31'),
            ('2018-09-01', '2018-09-30'),
            ('2018-10-01', '2018-10-31'),
            ('2018-11-01', '2018-11-30')
        ],
        2019: [
            ('2019-08-01', '2019-08-31'),
            ('2019-09-01', '2019-09-30'),
            ('2019-10-01', '2019-10-31'),
            ('2019-11-01', '2019-11-30')
        ],
        2020: [
            ('2020-08-01', '2020-08-31'),
            ('2020-09-01', '2020-09-30'),
            ('2020-10-01', '2020-10-31'),
            ('2020-11-01', '2020-11-30')
        ],
        2023: [
            ('2023-08-01', '2023-08-31'),
            ('2023-09-01', '2023-09-30'),
            ('2023-10-01', '2023-10-31'),
            ('2023-11-01', '2023-11-30')
        ]
    }

    indices = ['NDVI']  # Add more indices as you wish

    # Assuming 'shp_data' is a dictionary with years as keys and GeoDataFrames as values
    # You need to define these GeoDataFrames (shp_2018, shp_2019, shp_2020, shp_2023) before this point
    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(5, 10))

Processing period: 2018-08-01 to 2018-08-31
No images found for the period 2018-08-01 to 2018-08-31
Processing period: 2018-09-01 to 2018-09-30
No images found for the period 2018-09-01 to 2018-09-30
Processing period: 2018-10-01 to 2018-10-31
No images found for the period 2018-10-01 to 2018-10-31
Processing period: 2018-11-01 to 2018-11-30
No images found for the period 2018-11-01 to 2018-11-30


ValueError: No images were processed successfully

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 19 23:39:24 2024

@author: Janet.Mutuku

This script processes land cover data for various years using Google Earth Engine (GEE) and outputs the results as shapefiles.
"""

import ee
import time
import random
import os
from shapely.geometry import Polygon, MultiPolygon

# Initialize Earth Engine
ee.Initialize()

class PlanetNICFIProcessor:
    def __init__(self, year, date_ranges, indices, data_fc):
        self.year = year
        self.date_ranges = date_ranges
        self.indices = indices
        self.data_fc = data_fc

    def calculate_ndvi(self, image):
        return image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))

    def normalize_bands(self, image, band_names, geometry):
        def normalize_band(band_name):
            band = image.select(band_name).toFloat()
            min_max = band.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=5,
                maxPixels=1e13
            )
            min_val = ee.Number(min_max.get(ee.String(band_name).cat('_min')))
            max_val = ee.Number(min_max.get(ee.String(band_name).cat('_max')))
            return band.subtract(min_val).divide(max_val.subtract(min_val)).rename(ee.String(band_name).cat('_norm'))

        normalized_bands = [normalize_band(band) for band in band_names]
        return image.addBands(ee.Image.cat(normalized_bands))

    def process_all_months(self):
        processed_images = []

        # Use the entire year instead of specific months
        start_date = f"{self.year}-01-01"
        end_date = f"{self.year}-12-31"
        print(f"Processing period: {start_date} to {end_date}")

        nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
            .filterBounds(self.data_fc.geometry()) \
            .filterDate(start_date, end_date) \
            .map(self.calculate_ndvi)

        # Check if the collection is empty
        count = nicfi_planet.size().getInfo()
        if count == 0:
            print(f"No images found for the year {self.year}")
            print(f"Geometry bounds: {self.data_fc.geometry().bounds().getInfo()}")
            return None

        print(f"Found {count} images for the year {self.year}")

        # Print the bands of the first image
        first_image = ee.Image(nicfi_planet.first())
        print(f"Bands in the original image: {first_image.bandNames().getInfo()}")

        normalized_collection = nicfi_planet.map(lambda image: self.normalize_bands(image, ['B', 'G', 'R', 'N'], self.data_fc.geometry()))

        # Check the bands after normalization
        first_normalized = ee.Image(normalized_collection.median())
        print(f"Bands after normalization: {first_normalized.bandNames().getInfo()}")

        with_ndvi = normalized_collection#.map(self.calculate_ndvi)

        bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

        composite = with_ndvi.median().select(bands)

        # Check if all required bands are present
        available_bands = composite.bandNames().getInfo()
        missing_bands = [band for band in bands if band not in available_bands]
        if missing_bands:
            print(f"Warning: The following bands are missing: {missing_bands}")
            # Only select available bands
            bands = [band for band in bands if band in available_bands]

        if not bands:
            print("Error: No required bands are available in the composite image")
            return None

        processed_images.append(composite)

        if not processed_images:
            print("No images were processed successfully")
            return None

        return ee.Image.cat(processed_images)

    def retry_function(self, func, max_retries=5, initial_delay=1, factor=2):
        """Retry a function with exponential backoff."""
        retries = 0
        while retries < max_retries:
            try:
                return func()
            except ee.ee_exception.EEException as e:
                if "Too many concurrent aggregations" in str(e):
                    retries += 1
                    if retries == max_retries:
                        raise
                    delay = initial_delay * (factor ** retries) + random.uniform(0, 1)
                    print(f"Rate limit hit. Retrying in {delay:.2f} seconds...")
                    time.sleep(delay)
                else:
                    raise

    def add_bands_to_fc(self, image):
        def sample_image(feature):
            values = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=5,
                maxPixels=1e13
            )
            return feature.set(values)

        return self.data_fc.map(sample_image)

    def export_to_drive(self, description, file_format='SHP', folder='data_with_bands_planet_ncifi'):
        combined_image = self.process_all_months()
        if combined_image is None:
            print(f"No data to export for {description}")
            return

        data_with_bands = self.add_bands_to_fc(combined_image)

        task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format,
            folder=folder
        )
        task.start()
        print(f"Started export task: {task.id}")
        print("Check your Earth Engine Tasks panel to monitor progress.")

class SubclassProcessor(PlanetNICFIProcessor):
    def __init__(self, subclass_df, year, date_ranges, indices, subclass_name):
        data_fc = gdf_to_ee_feature_collection(subclass_df)
        super().__init__(year, date_ranges, indices, data_fc)
        self.subclass_name = subclass_name

def gdf_to_ee_feature_collection(gdf):
    features = []
    for i, row in gdf.iterrows():
        geometry = row.geometry
        if isinstance(geometry, Polygon):
            ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        elif isinstance(geometry, MultiPolygon):
            polygons = []
            for polygon in geometry.geoms:
                polygons.append(list(polygon.exterior.coords))
            ee_geometry = ee.Geometry.MultiPolygon(polygons)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geometry)}")

        properties = row.drop('geometry').fillna(0).to_dict()  # Replace NaN with 0
        feature = ee.Feature(ee_geometry, properties)
        features.append(feature)

    return ee.FeatureCollection(features)

# Main execution
if __name__ == "__main__":
    # Set up your Google Drive folder
    os.chdir('/content/drive/MyDrive/Crop Monitoring/crop_monitoring/crop_types_data/data_with_bands_planet_ncifi')

    # Years to process
    years = [2018, 2019, 2020, 2023]

    # Define date ranges for each year (now using the entire year)
    date_ranges = {year: [(f"{year}-01-01", f"{year}-12-31")] for year in years}

    indices = ['NDVI']  # Add more indices as you wish

    # Assuming 'shp_data' is a dictionary with years as keys and GeoDataFrames as values
    # You need to define these GeoDataFrames (shp_2018, shp_2019, shp_2020, shp_2023) before this point
    shp_data = {
        2018: shp_2018,
        2019: shp_2019,
        2020: shp_2020,
        2023: shp_2023
    }

    # Process each year and each subclass separately
    for year in years:
        for subclass_name, subclass_df in shp_data[year].groupby('Sub_class'):
            processor = SubclassProcessor(subclass_df, year, date_ranges[year], indices, subclass_name)
            try:
                # Ensure the description is a string and doesn't contain any special formatting characters
                description = f"data_{year}_with_bands_subclass_{subclass_name}"
                description = description.replace("%", "").replace("{", "").replace("}", "")
                processor.export_to_drive(description)
            except ee.ee_exception.EEException as e:
                print(f"Error processing {year}, subclass {subclass_name}: {str(e)}")

            # Add a delay between processing subclasses
            time.sleep(random.uniform(5, 10))

Processing period: 2018-01-01 to 2018-12-31
Found 2 images for the year 2018
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']
Bands after normalization: ['B', 'G', 'R', 'N', 'NDVI', 'B_norm', 'G_norm', 'R_norm', 'N_norm']
Started export task: UBWQDZP4UJ7WOGSDQD7EK2TJ
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-01-01 to 2018-12-31
Found 2 images for the year 2018
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']
Bands after normalization: ['B', 'G', 'R', 'N', 'NDVI', 'B_norm', 'G_norm', 'R_norm', 'N_norm']
Started export task: XFY3J5XSXT2PUV5MBBBTQF62
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2018-01-01 to 2018-12-31
Found 2 images for the year 2018
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']
Bands after normalization: ['B', 'G', 'R', 'N', 'NDVI', 'B_norm', 'G_norm', 'R_norm', 'N_norm']
Started export task: NN4SDJU7YPFYOFW7KC5R4ECU
Check your Earth Engine Tasks panel to monitor progress

Error processing 2020, subclass bare_built_up: Too many concurrent aggregations.
Processing period: 2020-01-01 to 2020-12-31
Found 5 images for the year 2020
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']


Error processing 2020, subclass cereals: The service is currently unavailable.
Processing period: 2020-01-01 to 2020-12-31
Found 5 images for the year 2020
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']


Error processing 2020, subclass fallow: Too many concurrent aggregations.
Processing period: 2020-01-01 to 2020-12-31
Found 5 images for the year 2020
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']


Error processing 2020, subclass legumes: Too many concurrent aggregations.
Processing period: 2020-01-01 to 2020-12-31
Found 5 images for the year 2020
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']


...

Error processing 2023, subclass cereals: Too many concurrent aggregations.
Processing period: 2023-01-01 to 2023-12-31
Found 12 images for the year 2023
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']


Error processing 2023, subclass fallow: Too many concurrent aggregations.
Processing period: 2023-01-01 to 2023-12-31
Found 12 images for the year 2023
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']


Error processing 2023, subclass legumes: Too many concurrent aggregations.
Processing period: 2023-01-01 to 2023-12-31
Found 12 images for the year 2023
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']
Bands after normalization: ['B', 'G', 'R', 'N', 'NDVI', 'B_norm', 'G_norm', 'R_norm', 'N_norm']
Started export task: XXUGTHSQ67HUTXU4RWOA3NPH
Check your Earth Engine Tasks panel to monitor progress.
Processing period: 2023-01-01 to 2023-12-31
Found 12 images for the year 2023
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']


Error processing 2023, subclass tree_crops: Too many concurrent aggregations.
Processing period: 2023-01-01 to 2023-12-31
Found 12 images for the year 2023
Bands in the original image: ['B', 'G', 'R', 'N', 'NDVI']


Error processing 2023, subclass vegetables: Too many concurrent aggregations.


In [ ]:
# Import necessary librari
from google.colab import drive
def rename_property(feature, old_name, new_name):
    new_feature = feature.set(new_name, feature.get(old_name)).copyProperties(feature).set(old_name, None)
    return new_feature

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Load Sentinel-2 imagery and filter by date and bounds of the polygons
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(data_2020).filterDate('2020-09-01', '2020-09-28') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
    .map(calculate_ndvi)

# Select bands to use for classification
bands = ['B2', 'B3', 'B4', 'B8', 'NDVI']

# Reduce the image collection to median to get a single composite image
image = sentinel2.median().select(bands)

# Function to sample data from the image and integrate with feature collection
def add_bands_to_fc(fc, image):
    def add_bands(feature):
        sampled = image.reduceRegion(           #Reduce the image collection to get a median composite image.
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=10,
            maxPixels=1e13
        )
        return feature.set(sampled)   #Create a function to add bands to the feature collection by sampling the image.
    return fc.map(add_bands)

# Integrate the bands into the feature collection
data_2020_with_bands = add_bands_to_fc(data_2020, image)  # using teh cleaned data -- can change to the original data

#Convert FeatureCollection to pandas DataFrame
def fc_to_df(fc):
    features = fc.getInfo()['features']
    dict_list = [f['properties'] for f in features]
    df = pd.DataFrame(dict_list)
    return df

# Display the first 5 rows of the dataframe
df = fc_to_df(data_2020_with_bands)
print(df.head())

# Export the new shapefile dataset to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=data_2020_with_bands,
    description='data_2020_with_bands_sep',
    fileFormat='SHP'
)

export_task.start()

# Monitor the task status
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export task completed with status:', export_task.status())

In [ ]:
# Function to get feature properties
def get_properties(feature):
    return ee.Feature(None, feature.toDictionary())

# Map the function over the collection
properties_list = data_2018.map(get_properties).getInfo()

# Extract properties from the list of features
properties_list = [feature['properties'] for feature in properties_list['features']]

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(properties_list)

# Rename columns if needed
#df.rename(columns={'Annee': 'Year'}, inplace=True) #df.rename(columns={'old_column_name': 'new_column_name'}, inplace=True)


# Display the DataFrame
df.head()

In [ ]:
# @title Crop_Ncrop

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Crop_Ncrop').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Average Area by Crop Type

df.groupby('Speculatio')['Sup_ha'].mean().plot(kind='bar')

In [ ]:
# @title Admin1

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Admin1').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# Check for NaN values in each column
nan_counts = df.isna().sum()

# Display the columns with their respective NaN counts
print(nan_counts)

In [ ]:
# Display only columns with NaN values
nan_columns = nan_counts[nan_counts > 0]
print(nan_columns)


In [ ]:
# @title Default title text
# Define a function to add Earth Engine vector data as a layer to a Folium map
def add_ee_vector_layer(feature_collection, style, layer_name, map_object):
    painted = ee.Image().paint(feature_collection, 'constant', 2)  # Here 'constant' is a dummy property for visualization
    map_id_dict = painted.getMapId(style)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=layer_name,
        overlay=True,
        control=True
    ).add_to(map_object)

# Create a Folium map object
center = [14.4974, -14.4524]  # Center of the map (e.g., Senegal)
m1 = folium.Map(location=center, zoom_start=7)

# Styling for the vector layer
style = {
    'color': 'blue',  # Line color
    'fillColor': '00000000',  # Fill color with opacity (00)
}

# Add the merged crop fields to the map
add_ee_vector_layer(data_2018, style, 'Merged Crops 2019-2020', m1)

# Add a layer control panel to the map
folium.LayerControl().add_to(m1)

# Display the map
m1


### Polygons for FY2023

In [ ]:
print(vegetables.size().getInfo())

In [ ]:
print(sorgh.size().getInfo())

Next steps:
- create shape file with bands and NDVI
- edit the clumen class of crop, fallow, N-crop etc
- export the classes and bands in to Google earht engine asset
- and test the new asset if band values and added class are in GEE ASSET

### Normalized bands : integrate the bands in to shapfle (Vector) 📂

To normalize the spectral bands, we'll create a function that normalizes each band by subtracting the minimum value and dividing by the range (max - min) of that band. We'll apply this function to each band in the image collection before calculating the NDVI and proceeding with the rest of the steps.

### Senti2

In [ ]:
# Function to rename a property in a feature
def rename_property(feature, old_name, new_name):
    new_feature = feature.set(new_name, feature.get(old_name)).copyProperties(feature).set(old_name, None)
    return new_feature

# Function to normalize a band
def normalize_band(image, band_name, geometry):
    band = image.select(band_name)
    min_val = ee.Number(band.reduceRegion(ee.Reducer.min(), geometry, scale=10).get(band_name))
    max_val = ee.Number(band.reduceRegion(ee.Reducer.max(), geometry, scale=10).get(band_name))
    normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).float()
    return normalized_band.rename(band_name + '_norm')

# Function to normalize all specified bands
def normalize_bands(image, band_names, geometry):
    normalized_bands = [normalize_band(image, band_name, geometry) for band_name in band_names]
    return image.addBands(normalized_bands, overwrite=True)

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Load Sentinel-2 imagery and filter by date and bounds of the polygons
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(gnut.geometry()) \
    .filterDate('2023-08-10', '2023-10-10') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(lambda image: normalize_bands(image, ['B2', 'B3', 'B4', 'B8'], gnut.geometry())) \
    .map(calculate_ndvi)

# Select bands to use for classification
bands = ['B2_norm', 'B3_norm', 'B4_norm', 'B8_norm', 'NDVI', 'B2', 'B3', 'B4', 'B8']

# Reduce the image collection to median to get a single composite image
image = sentinel2.median().select(bands)

# Function to sample data from the image and integrate with feature collection
def add_bands_to_fc(fc, image):
    def add_bands(feature):
        # Sample the image at the feature's geometry
        sampled = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=10,
            maxPixels=1e13,
            bestEffort=True
        )
        # Convert the sampled dictionary to ensure all keys have non-null values
        sampled = sampled.map(lambda k, v: ee.Algorithms.If(v, v, 0))

        # Ensure that sampled values are added correctly to the feature
        return feature.set(sampled)

    # Apply the add_bands function to each feature in the feature collection
    return fc.map(add_bands)

# Integrate the bands into the feature collection
crop_with_bands = add_bands_to_fc(gnut, image)

# Add new properties to each feature
def add_properties(feature):
    return feature.set('class', 'crop').set('sub_class', 'legume').set('year', '2023').set('region', '???') #Annee = year, Sub-class = Type, Crop_Ncrop = class

merged2023_class_with_properties = crop_with_bands.map(add_properties)

# Rename the property "Speculat" to "Speculatio"
def rename_speculat_to_speculatio(feature):
    return rename_property(feature, 'Speculat', 'name') #name =Speculatio)

merged2023_class_with_properties = merged2023_class_with_properties.map(rename_speculat_to_speculatio)

# Get the first few features to display their properties
features_to_display = merged2023_class_with_properties.limit(5).getInfo()['features']

# Extract properties into a list of dictionaries
properties_list = [feature['properties'] for feature in features_to_display]

# Create a DataFrame
df = pd.DataFrame(properties_list)

# Display the DataFrame
print(df.tail())

# Debugging: Check the contents before export
print("Feature collection before export: ", merged2023_class_with_properties.getInfo())

# Export the modified Feature Collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection=merged2023_class_with_properties,
    description='Export Crop_normalized_2023 with classes',
    assetId='projects/ee-kkidia3/assets/groundnut_class_final_x'
)

export_task.start()

print("Export task started. Check the Earth Engine Code Editor for task status.")


In [ ]:

#sub-classs
cereals2 = soye.merge(millet).merge(milletmix).merge(sorgh).merge(fonio).merge(maize).merge(millet).merge(sesame).merge(wheat).merge(rice) #class = crop, and sub_class = cereals
legumes = gnut.merge(gnutmix).merge(cowpea).merge(cowpmix).merge(vouandzou) #class = crop, and sub_class = legumes
vegetables = cassava.merge(eggplant).merge(gsorrel).merge(okra).merge(potato).merge(squash).merge(taro).merge(melon) #class = crop, and sub_class = vegetables
fallow #as class and sub-class
tree_crops = cashew #as class and sub-class
#fallow = fallow.filter(ee.Filter.eq('class', 'Fallow')) #.merge(fallow) is not a crop

### NICFI

### Export all Classes and Sub-Classes as one SHP with band+ classes _NICFI Data

Fallow

In [ ]:
# Function to rename a property in a feature
def rename_property(feature, old_name, new_name):
    new_feature = feature.set(new_name, feature.get(old_name)).copyProperties(feature).set(old_name, None)
    return new_feature

# Function to normalize a band
def normalize_band(image, band_name, geometry):
    band = image.select(band_name)
    min_val = ee.Number(band.reduceRegion(ee.Reducer.min(), geometry, scale=10).get(band_name))
    max_val = ee.Number(band.reduceRegion(ee.Reducer.max(), geometry, scale=10).get(band_name))
    normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).float()
    return normalized_band.rename(band_name + '_norm')

# Function to normalize all specified bands
def normalize_bands(image, band_names, geometry):
    normalized_bands = [normalize_band(image, band_name, geometry) for band_name in band_names]
    return image.addBands(normalized_bands, overwrite=True)

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
    return image.addBands(ndvi)

# Load NICFI/Planet imagery and filter by date and bounds of the polygons
nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
    .filterBounds(fallow.geometry()) \
    .filterDate('2023-09-01', '2023-09-30') \
    .map(lambda image: normalize_bands(image, ['B', 'G', 'R', 'N'], fallow.geometry())) \
    .map(calculate_ndvi)

# Select bands to use for classification
bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

# Reduce the image collection to median to get a single composite image
image = nicfi_planet.median().select(bands)

# Function to sample data from the image and integrate with feature collection
def add_bands_to_fc(fc, image):
    def add_bands(feature):
        # Sample the image at the feature's geometry
        sampled = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=5,
            maxPixels=1e13,
            bestEffort=True
        )
        # Convert the sampled dictionary to ensure all keys have non-null values
        sampled = sampled.map(lambda k, v: ee.Algorithms.If(v, v, 0))

        # Ensure that sampled values are added correctly to the feature
        return feature.set(sampled)

    # Apply the add_bands function to each feature in the feature collection
    return fc.map(add_bands)

# Integrate the bands into the feature collection
crop_with_bands = add_bands_to_fc(fallow, image)

# Add new properties to each feature
def add_properties(feature):
    return feature.set('Class', 'Fallow').set('Sub_class', 'Fallow').set('Year', '2023').set('Region', '???') #Annee = year, Sub-class = Type, Crop_Ncrop = class

merged2023_class_with_properties = crop_with_bands.map(add_properties)

# Rename the property "Speculat" to "Speculatio"
def rename_speculat_to_speculatio(feature):
    return rename_property(feature, 'Specult', 'Name') #name =Specult

merged2023_class_with_properties = merged2023_class_with_properties.map(rename_speculat_to_speculatio)

# Get the first few features to display their properties
features_to_display = merged2023_class_with_properties.limit(5).getInfo()['features']

# Extract properties into a list of dictionaries
properties_list = [feature['properties'] for feature in features_to_display]

# Create a DataFrame
df = pd.DataFrame(properties_list)

# Display the DataFrame
print(df.tail())

# Debugging: Check the contents before export
print("Feature collection before export: ", merged2023_class_with_properties.getInfo())

# Export the modified Feature Collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection=merged2023_class_with_properties,
    description='Export Crop_normalized_2023 with classes',
    assetId='projects/ee-kkidia3/assets/sept_fallow_class_nicfi'
)

export_task.start()

print("Export task started. Check the Earth Engine Code Editor for task status.")


Vegitables

In [ ]:
# Function to rename a property in a feature
def rename_property(feature, old_name, new_name):
    new_feature = feature.set(new_name, feature.get(old_name)).copyProperties(feature).set(old_name, None)
    return new_feature

# Function to normalize a band
def normalize_band(image, band_name, geometry):
    band = image.select(band_name)
    min_val = ee.Number(band.reduceRegion(ee.Reducer.min(), geometry, scale=10).get(band_name))
    max_val = ee.Number(band.reduceRegion(ee.Reducer.max(), geometry, scale=10).get(band_name))
    normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).float()
    return normalized_band.rename(band_name + '_norm')

# Function to normalize all specified bands
def normalize_bands(image, band_names, geometry):
    normalized_bands = [normalize_band(image, band_name, geometry) for band_name in band_names]
    return image.addBands(normalized_bands, overwrite=True)

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
    return image.addBands(ndvi)

# Load NICFI/Planet imagery and filter by date and bounds of the polygons
nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
    .filterBounds(vegetables.geometry()) \
    .filterDate('2023-09-01', '2023-09-30')  \
    .map(lambda image: normalize_bands(image, ['B', 'G', 'R', 'N'], vegetables.geometry())) \
    .map(calculate_ndvi)

# Select bands to use for classification
bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

# Reduce the image collection to median to get a single composite image
image = nicfi_planet.median().select(bands)

# Function to sample data from the image and integrate with feature collection
def add_bands_to_fc(fc, image):
    def add_bands(feature):
        # Sample the image at the feature's geometry
        sampled = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=5,
            maxPixels=1e13,
            bestEffort=True
        )
        # Convert the sampled dictionary to ensure all keys have non-null values
        sampled = sampled.map(lambda k, v: ee.Algorithms.If(v, v, 0))

        # Ensure that sampled values are added correctly to the feature
        return feature.set(sampled)

    # Apply the add_bands function to each feature in the feature collection
    return fc.map(add_bands)

# Integrate the bands into the feature collection
crop_with_bands = add_bands_to_fc(vegetables, image)

# Add new properties to each feature
def add_properties(feature):
    return feature.set('Class', 'Crop').set('Sub_class', 'Vegetables').set('Year', '2023').set('Region', '???') #Annee = year, Sub-class = Type, Crop_Ncrop = class

merged2023_class_with_properties = crop_with_bands.map(add_properties)

# Rename the property "Speculat" to "Speculatio"
def rename_speculat_to_speculatio(feature):
    return rename_property(feature, 'Speculat', 'Name') #name =Specult)

merged2023_class_with_properties = merged2023_class_with_properties.map(rename_speculat_to_speculatio)

# Get the first few features to display their properties
features_to_display = merged2023_class_with_properties.limit(5).getInfo()['features']

# Extract properties into a list of dictionaries
properties_list = [feature['properties'] for feature in features_to_display]

# Create a DataFrame
df = pd.DataFrame(properties_list)

# Display the DataFrame
print(df.tail())

# Debugging: Check the contents before export
print("Feature collection before export: ", merged2023_class_with_properties.getInfo())

# Export the modified Feature Collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection=merged2023_class_with_properties,
    description='Export Crop_normalized_2023 with classes',
    assetId='projects/ee-kkidia3/assets/sept_vegetables_class_nicfi'
)

export_task.start()

print("Export task started. Check the Earth Engine Code Editor for task status.")

legumes

In [ ]:
# Function to rename a property in a feature
def rename_property(feature, old_name, new_name):
    new_feature = feature.set(new_name, feature.get(old_name)).copyProperties(feature).set(old_name, None)
    return new_feature

# Function to normalize a band
def normalize_band(image, band_name, geometry):
    band = image.select(band_name)
    min_val = ee.Number(band.reduceRegion(ee.Reducer.min(), geometry, scale=10).get(band_name))
    max_val = ee.Number(band.reduceRegion(ee.Reducer.max(), geometry, scale=10).get(band_name))
    normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).float()
    return normalized_band.rename(band_name + '_norm')

# Function to normalize all specified bands
def normalize_bands(image, band_names, geometry):
    normalized_bands = [normalize_band(image, band_name, geometry) for band_name in band_names]
    return image.addBands(normalized_bands, overwrite=True)

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
    return image.addBands(ndvi)

# Load NICFI/Planet imagery and filter by date and bounds of the polygons
nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
    .filterBounds(legumes.geometry()) \
    .filterDate('2023-09-01', '2023-09-30') \
    .map(lambda image: normalize_bands(image, ['B', 'G', 'R', 'N'], legumes.geometry())) \
    .map(calculate_ndvi)

# Select bands to use for classification
bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

# Reduce the image collection to median to get a single composite image
image = nicfi_planet.median().select(bands)

# Function to sample data from the image and integrate with feature collection
def add_bands_to_fc(fc, image):
    def add_bands(feature):
        sampled = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=5,
            maxPixels=1e13,
            bestEffort=True
        )
        sampled = sampled.map(lambda k, v: ee.Algorithms.If(v, v, 0))
        return feature.set(sampled)

    return fc.map(add_bands)

# Integrate the bands into the feature collection
crop_with_bands = add_bands_to_fc(legumes, image)

# Add new properties to each feature
def add_properties(feature):
    return feature.set('Class', 'Crop').set('Sub_class', 'Legumes').set('Year', '2023').set('Region', '???')

merged2023_class_with_properties = crop_with_bands.map(add_properties)

# Rename the property "Speculat" to "Speculatio"
def rename_speculat_to_speculatio(feature):
    return rename_property(feature, 'Speculat', 'Name')

merged2023_class_with_properties = merged2023_class_with_properties.map(rename_speculat_to_speculatio)

# List of properties to remove
properties_to_remove = [
    "altitudeMo", "area", "begin", "descriptio", "drawOrder", "end",
    "extrude", "icon", "id", "layer", "path", "perimeter", "system_ind",
    "tessellate", "timestamp", "visibility"
]

# Function to remove specified properties from the feature collection
def remove_properties(data, properties_to_remove):
    def clean_feature(feature):
        for prop in properties_to_remove:
            feature = feature.set(prop, None)
        return feature

    return data.map(clean_feature)

# Remove the unwanted properties
final_data = remove_properties(merged2023_class_with_properties, properties_to_remove)

# Function to rename the 'Name' property value based on given rules
def rename_name_property(feature):
    name = ee.String(feature.get('Name'))
    new_name = ee.Algorithms.If(name.equals('goundnut_mixed'), 'groundnut_mixed', name)
    new_name = ee.Algorithms.If(ee.String(new_name).equals('mixed_cowpea'), 'cowpea_mixed', new_name)
    return feature.set('Name', new_name)

# Apply the renaming rule to 'Name' property
final_data = final_data.map(rename_name_property)

# Get the first few features to display their properties
features_to_display = final_data.limit(5).getInfo()['features']

# Extract properties into a list of dictionaries
properties_list = [feature['properties'] for feature in features_to_display]

# Create a DataFrame
df = pd.DataFrame(properties_list)

# Display the DataFrame
print(df.tail())

# Debugging: Check the contents before export
print("Feature collection before export: ", final_data.getInfo())

# Export the modified Feature Collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection=final_data,
    description='Export Crop_normalized_2023 with classes',
    assetId='projects/ee-kkidia3/assets/sept_legumes_class_nicfi'
)

export_task.start()


Cereals2 has mutimple unnecessary properties to be removed

In [ ]:
# Function to rename a property in a feature
def rename_property(feature, old_name, new_name):
    new_feature = feature.set(new_name, feature.get(old_name)).copyProperties(feature).set(old_name, None)
    return new_feature

# Function to normalize a band
def normalize_band(image, band_name, geometry):
    band = image.select(band_name)
    min_val = ee.Number(band.reduceRegion(ee.Reducer.min(), geometry, scale=10).get(band_name))
    max_val = ee.Number(band.reduceRegion(ee.Reducer.max(), geometry, scale=10).get(band_name))
    normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).float()
    return normalized_band.rename(band_name + '_norm')

# Function to normalize all specified bands
def normalize_bands(image, band_names, geometry):
    normalized_bands = [normalize_band(image, band_name, geometry) for band_name in band_names]
    return image.addBands(normalized_bands, overwrite=True)

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
    return image.addBands(ndvi)

# Load NICFI/Planet imagery and filter by date and bounds of the polygons
nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
    .filterBounds(cereals2.geometry()) \
    .filterDate('2023-09-01', '2023-09-30')  \
    .map(lambda image: normalize_bands(image, ['B', 'G', 'R', 'N'], cereals2.geometry())) \
    .map(calculate_ndvi)

# Select bands to use for classification
bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

# Reduce the image collection to median to get a single composite image
image = nicfi_planet.median().select(bands)

# Function to sample data from the image and integrate with feature collection
def add_bands_to_fc(fc, image):
    def add_bands(feature):
        # Sample the image at the feature's geometry
        sampled = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=5, #5m for NICFI
            maxPixels=1e13,
            bestEffort=True
        )
        # Convert the sampled dictionary to ensure all keys have non-null values
        sampled = sampled.map(lambda k, v: ee.Algorithms.If(v, v, 0))

        # Ensure that sampled values are added correctly to the feature
        return feature.set(sampled)

    # Apply the add_bands function to each feature in the feature collection
    return fc.map(add_bands)

# Integrate the bands into the feature collection
crop_with_bands = add_bands_to_fc(cereals2, image)

# Add new properties to each feature
def add_properties(feature):
    return feature.set('Class', 'Crop').set('Sub_class', 'Cereals').set('Year', '2023').set('Region', '???') #Annee = year, Sub-class = Type, Crop_Ncrop = class

merged2023_class_with_properties = crop_with_bands.map(add_properties)

# Rename properties to "Name"
def rename_to_name(feature):
    feature = rename_property(feature, 'Speculat', 'Name')
    feature = rename_property(feature, 'Specult', 'Name')
    feature = rename_property(feature, 'Speculatio', 'Name')
    #feature = rename_property(feature, 'Name', 'Name')
    return feature

merged2023_class_with_properties = merged2023_class_with_properties.map(rename_to_name)

# Specify the properties to retain (remove some unnecessary properties)
properties_to_retain = ['Class', 'Sub_class', 'Year', 'Region', 'Name', 'B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N','id' ]

# Select only the specified properties
filtered_features = merged2023_class_with_properties.select(properties_to_retain)

# Get the first few features to display their properties
features_to_display = filtered_features.limit(5).getInfo()['features']

# Extract properties into a list of dictionaries
properties_list = [feature['properties'] for feature in features_to_display]

# Create a DataFrame
df = pd.DataFrame(properties_list)

# Display the DataFrame
print(df.tail())

# Debugging: Check the contents before export
print("Feature collection before export: ", filtered_features.getInfo())

# Export the modified Feature Collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection=filtered_features,
    description='Export Crop_normalized_2023 with classes',
    assetId='projects/ee-kkidia3/assets/sept_cereals_class_nicfi'
)

export_task.start()

print("Export task started. Check the Earth Engine Code Editor for task status.")


Tree_Crops

In [ ]:
# Function to rename a property in a feature
def rename_property(feature, old_name, new_name):
    new_feature = feature.set(new_name, feature.get(old_name)).copyProperties(feature).set(old_name, None)
    return new_feature

# Function to normalize a band
def normalize_band(image, band_name, geometry):
    band = image.select(band_name)
    min_val = ee.Number(band.reduceRegion(ee.Reducer.min(), geometry, scale=10).get(band_name))
    max_val = ee.Number(band.reduceRegion(ee.Reducer.max(), geometry, scale=10).get(band_name))
    normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).float()
    return normalized_band.rename(band_name + '_norm')

# Function to normalize all specified bands
def normalize_bands(image, band_names, geometry):
    normalized_bands = [normalize_band(image, band_name, geometry) for band_name in band_names]
    return image.addBands(normalized_bands, overwrite=True)

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
    return image.addBands(ndvi)

# Load NICFI/Planet imagery and filter by date and bounds of the polygons
nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
    .filterBounds(tree_crops.geometry()) \
    .filterDate('2023-09-01', '2023-09-30')  \
    .map(lambda image: normalize_bands(image, ['B', 'G', 'R', 'N'], tree_crops.geometry())) \
    .map(calculate_ndvi)

# Select bands to use for classification
bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

# Reduce the image collection to median to get a single composite image
image = nicfi_planet.median().select(bands)

# Function to sample data from the image and integrate with feature collection
def add_bands_to_fc(fc, image):
    def add_bands(feature):
        # Sample the image at the feature's geometry
        sampled = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=5,
            maxPixels=1e13,
            bestEffort=True
        )
        # Convert the sampled dictionary to ensure all keys have non-null values
        sampled = sampled.map(lambda k, v: ee.Algorithms.If(v, v, 0))

        # Ensure that sampled values are added correctly to the feature
        return feature.set(sampled)

    # Apply the add_bands function to each feature in the feature collection
    return fc.map(add_bands)

# Integrate the bands into the feature collection
crop_with_bands = add_bands_to_fc(tree_crops, image)

# Add new properties to each feature
def add_properties(feature):
    return feature.set('Class', 'Tree_crops').set('Sub_class', 'Tree_crops').set('Year', '2023').set('Region', '???') #Annee = year, Sub-class = Type, Crop_Ncrop = class

merged2023_class_with_properties = crop_with_bands.map(add_properties)

# Rename the property "Speculat" to "Speculatio"
def rename_speculat_to_speculatio(feature):
    return rename_property(feature, 'Speculat', 'Name') #name =Speculat)

merged2023_class_with_properties = merged2023_class_with_properties.map(rename_speculat_to_speculatio)

# Get the first few features to display their properties
features_to_display = merged2023_class_with_properties.limit(5).getInfo()['features']

# Extract properties into a list of dictionaries
properties_list = [feature['properties'] for feature in features_to_display]

# Create a DataFrame
df = pd.DataFrame(properties_list)

# Display the DataFrame
print(df.tail())

# Debugging: Check the contents before export
print("Feature collection before export: ", merged2023_class_with_properties.getInfo())

# Export the modified Feature Collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection=merged2023_class_with_properties,
    description='Export Crop_normalized_2023 with classes',
    assetId='projects/ee-kkidia3/assets/sept_tree_crops_class_nicfi'
)

export_task.start()

print("Export task started. Check the Earth Engine Code Editor for task status.")

### ALL 2023 Poylgons with NICFI Classes of bands and NDVI

In [ ]:
# Load individual rainfed 2023 crop feature collections
asset_2023 = {
    'tree_crops': ee.FeatureCollection('projects/ee-kkidia3/assets/tree_crops_nicfi_months'),
    'cereals': ee.FeatureCollection('projects/ee-kkidia3/assets/cereals_nicfi_months'),
    'legumes': ee.FeatureCollection('projects/ee-kkidia3/assets/legumes_nicfi_months'),
    'vegetables': ee.FeatureCollection('projects/ee-kkidia3/assets/vegetables_nicfi_months'),
    'fallow': ee.FeatureCollection('projects/ee-kkidia3/assets/fallow_nicfi_months')
}

# Merge the feature collections into a single collection
merged_collection = asset_2023['tree_crops'] \
    .merge(asset_2023['cereals']) \
    .merge(asset_2023['legumes']) \
    .merge(asset_2023['vegetables'])\
    .merge(asset_2023['fallow'])

# Define the export task
export_task = ee.batch.Export.table.toDrive(
    collection=merged_collection,
    description='sept_data_2023',
    fileFormat='geojson',
    folder='data_2023_class_nicfi',
    fileNamePrefix='sept_data_2023_2_class_nicfi'
)

# # Debugging: Check the contents before export
# print("Feature collection before export: ", merged2023_class_with_properties.getInfo())

# Export the modified Feature Collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection = merged_collection,
    description='data_2023_months',
    assetId='projects/ee-kkidia3/assets/data_2023_nicfi_months'
)

export_task.start()

print("Export task started. Check the Earth Engine Code Editor for task status.")

# Check the status of the export
while export_task.active():
    print('Exporting... Status: {}'.format(export_task.status()))
    time.sleep(30)  # Check the status every 30 seconds

print('Export completed. Status: {}'.format(export_task.status()))

### Renmae some features under "Name" Class

In [ ]:
# Load the feature collection
foo9= ee.FeatureCollection('projects/ee-kkidia3/assets/vegetables_class_nicfi')

# Function to rename the 'Name' property value based on given rules
def rename_name_property(feature):
    name = ee.String(feature.get('Name'))
    new_name = ee.Algorithms.If(name.equals('Okra'), 'okra', name) # in vegetables class
    # new_name = ee.Algorithms.If(name.equals('goundnut_mixed'), 'groundnut_mixed', name) ## in legumes sub class
    # new_name = ee.Algorithms.If(ee.String(new_name).equals('mixed_cowpea'), 'cowpea_mixed', new_name) ##  in legumes sub-class
    return feature.set('Name', new_name)
# Apply the renaming function to the feature collection
foo9 = foo9.map(rename_name_property)

# Export the modified Feature Collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection=foo9,
    description='Export_Crop_Renamed',
    assetId='projects/ee-kkidia3/assets/vegetables_class_nicfi_renamed' #renamed as legumes_class_nicfi
)
export_task.start()

print("Export task started. Check the Earth Engine Code Editor for task status.")

In [ ]:
# @title ALL 2023 Poylgons with NICFI Classes of bands and NDVI +++++++++++++++ 12 Months
foo = ee.FeatureCollection('projects/ee-kkidia3/assets/vegetables_class_nicfi_renamed')
##################################################################################################################################################
# Function to normalize a band
def normalize_band(image, band_name, geometry):
    band = image.select(band_name)
    min_val = ee.Number(band.reduceRegion(ee.Reducer.min(), geometry, scale=10).get(band_name))
    max_val = ee.Number(band.reduceRegion(ee.Reducer.max(), geometry, scale=10).get(band_name))
    normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).float()
    return normalized_band.rename(band_name + '_norm')

# Function to normalize all specified bands
def normalize_bands(image, band_names, geometry):
    normalized_bands = [normalize_band(image, band_name, geometry) for band_name in band_names]
    return image.addBands(normalized_bands, overwrite=True)

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
    return image.addBands(ndvi)

# Function to add bands to the feature collection
def add_bands_to_fc(fc, image, month):
    def add_bands(feature):
        sampled = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=5,
            maxPixels=1e13,
            bestEffort=True
        )
        sampled = sampled.map(lambda k, v: ee.Algorithms.If(v, v, 0))

        # Create month-specific properties
        keys = sampled.keys()
        values = keys.map(lambda k: sampled.get(k))
        month_sampled = ee.Dictionary.fromLists(keys.map(lambda k: ee.String(k).cat('_').cat(ee.Number(month).format())), values)

        return feature.setMulti(month_sampled)

    return fc.map(add_bands)

# Function to get the last day of a given month
def get_last_day_of_month(year, month):
    next_month = (month % 12) + 1
    next_month_year = year if month < 12 else year + 1
    last_day = ee.Date(f'{next_month_year}-{next_month:02d}-01').advance(-1, 'day').format('yyyy-MM-dd')
    return last_day

# Function to process data for a given month and year
def process_month(year, month):
    start_date = f'{year}-{month:02d}-01'
    end_date = get_last_day_of_month(year, month).getInfo()

    nicfi_planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
        .filterBounds(foo.geometry()) \
        .filterDate(start_date, end_date) \
        .map(lambda image: normalize_bands(image, ['B', 'G', 'R', 'N'], foo.geometry())) \
        .map(calculate_ndvi)

    bands = ['B_norm', 'G_norm', 'R_norm', 'N_norm', 'NDVI', 'B', 'G', 'R', 'N']

    image = nicfi_planet.median().select(bands)

    crop_with_bands = add_bands_to_fc(foo, image, month)

    return crop_with_bands

# Process data for each month in 2023 and merge into a single feature collection
merged_feature_collection = foo
for month in range(1, 13):
    monthly_fc = process_month(2023, month)
    merged_feature_collection = merged_feature_collection.map(
        lambda f: f.setMulti(monthly_fc.filter(ee.Filter.eq('system:index', f.get('system:index'))).first().toDictionary())
    )

    print(f'Processed month {month}...')

# Export the merged feature collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection=merged_feature_collection,
    description='Export_vegetables_months',
    assetId='projects/ee-kkidia3/assets/vegetables_class_nicfi_renamed_months'
    )


# # Export the merged feature collection to Google Drive as GeoJSON
# export_task = ee.batch.Export.table.toDrive(
#     collection=merged_feature_collection,
#     description='Export_Crop_normalized_2023_to_Drive',
#     fileFormat='GeoJSON',
#     folder='foo6',
#     fileNamePrefix='foo6'
# )
export_task.start()

print("Export task started. Check the Earth Engine Code Editor for task status.")



In [ ]:
# @title Test the exported GEE asset
xx = ee.FeatureCollection('projects/ee-kkidia3/assets/data_2023_nicfi_months')
# Get the first few features to display their properties
f_display =xx.limit(5000).getInfo()['features']  #Limited only 5000

# Crop_2023_norm_bands_class = load_crop_data('projects/ee-kkidia3/assets/Crop_2023_No_norm_bands_class')
# # Get the first few features to display their properties
# f_display =Crop_2023_norm_bands_class.limit(5).getInfo()['features']

# Extract properties into a list of dictionaries
p_list = [feature['properties'] for feature in f_display]
# Print the properties of the first few features
#for feature in features_to_display:
   # print(feature['properties'])

# Create a DataFrame from the properties list
df = pd.DataFrame(p_list)

df.head() #show it in data fram


,B,B_1,B_10,B_11,B_12,B_2,B_3,B_4,B_5,B_6,...,R_norm_4,R_norm_5,R_norm_6,R_norm_7,R_norm_8,R_norm_9,Region,Sub_class,Year,id
0,793.967797,562.901458,804.657593,594.241187,558.515172,514.074223,504.273836,515.258144,487.764391,431.006024,...,0.210879,0.203596,0.205572,0.240610,0.409707,0.395673,???,Tree_crops,2023,6.0
1,736.538466,575.151121,729.273378,591.958492,562.291627,529.610210,506.465410,523.047948,541.390506,515.292462,...,0.295722,0.382847,0.453650,0.122955,0.172472,0.336460,???,Tree_crops,2023,7.0
2,662.401625,465.788166,623.506855,444.602136,466.485234,447.104052,413.260734,463.249237,406.865309,406.954010,...,0.181106,0.181972,0.216503,0.257183,0.257825,0.224939,???,Tree_crops,2023,4.0
3,658.475114,532.622117,676.540495,547.975668,545.063343,502.563244,510.996202,562.685341,522.697409,531.518140,...,0.484181,0.532204,0.553644,0.566756,0.393534,0.105061,???,Tree_crops,2023,3.0
4,699.611807,574.250786,708.432271,590.548435,574.144886,537.637467,532.679715,594.283327,549.115129,552.693002,...,0.562260,0.596732,0.565190,0.659613,0.431805,0.219273,???,Tree_crops,2023,2.0


In [ ]:
# @title Name

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Name vs NDVI September

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Class'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='NDVI_9', y='Class', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
# @title Name vs NDVI November

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Class'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='NDVI_11', y='Class', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
# @title Mean NDVI by Crop Name

import matplotlib.pyplot as plt

mean_ndvi = df.groupby('Sub_class')['NDVI'].mean().sort_values()

plt.figure(figsize=(10, 6))
plt.bar(mean_ndvi.index, mean_ndvi.values)
plt.xlabel('Crop Name')
plt.ylabel('Mean NDVI')
plt.title('Mean NDVI by Crop Name')
_ = plt.xticks(rotation=45, ha='right')

In [ ]:
from matplotlib import pyplot as plt
_df_18['G_norm'].plot(kind='line', figsize=(8, 4), title='G_norm')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
print(xx.size().getInfo())

In [ ]:
# Check for NaN or None values in the DataFrame
null_values = df.isnull().sum()

# Filter properties that have NaN or None values
properties_with_nulls = null_values[null_values > 0]

# Display the properties with NaN or None values and their count
print(properties_with_nulls)

In [ ]:
# @title Name vs B_norm

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Class'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='B_norm', y='Class', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
from matplotlib import pyplot as plt
_df_28['B'].plot(kind='line', figsize=(8, 4), title='B')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
_df_30['G'].plot(kind='line', figsize=(8, 4), title='G')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# @title B

from matplotlib import pyplot as plt
df['B'].plot(kind='line', figsize=(8, 4), title='B')
plt.gca().spines[['top', 'right']].set_visible(False)

Sample randomforest calssification only gnut

Remove overlapping polygons from your merged feature collection. Overlaps is due to data collection

For example let me sort out only two polygons out of maney gnut polygons for closer look and simple analysis.

In [ ]:
# @title Default title text
# Select two specific polygons by their indices or properties. e.g gnut
# Here, I want to select the first two polygons.
poly_2 = gnut.toList(2)

#print(poly_2)

# Get the individual polygons
polygon1 = ee.Feature(poly_2.get(0))
polygon2 = ee.Feature(poly_2.get(1))

# Print the geometries of the selected polygons to verify.
print('Polygon 1_Blue:', polygon1.geometry().getInfo())
print('Polygon 2_Red:', polygon2.geometry().getInfo())


# Create a map centered at an arbitrary point @polygon1
map_center = [13.237198228125724,-14.715474917616827]  #set this to a blue selected @polygon1
m = folium.Map(location=map_center, zoom_start=30) #

# Define a function to add a feature to the folium map.
def add_feature_to_map(feature, map_obj, color):
    geom = feature.geometry().getInfo()
    coords = geom['coordinates']
    if geom['type'] == 'Polygon':
        folium.Polygon(locations=[(pt[1], pt[0]) for pt in coords[0]], color=color).add_to(map_obj)
    elif geom['type'] == 'MultiPolygon':
        for poly in coords:
            folium.Polygon(locations=[(pt[1], pt[0]) for pt in poly[0]], color=color).add_to(map_obj)

# Add polygons to the map
add_feature_to_map(polygon1, m, 'blue')
add_feature_to_map(polygon2, m, 'red')

# Display the map
m

In [ ]:
# Calculate the total area of polygons in square meters for Rainfed season 2023. Example groundnut
area = gnut.geometry().area().format('%.1f').getInfo()
print(f"Area M2: {area} square meters OR,")

# Calculate the area in hectares (1 hectare = 10,000 square meters)
area_ha = gnut.geometry().area().divide(10000).format('%.1f').getInfo() # Convert square meters to hectares
print(f"Area Ha: {area_ha} hectares")

# Calculate the perimeter in meters
perimeter = gnut.geometry().perimeter().format('%.1f').getInfo()
print(f"Perimeter: {perimeter} meters")

# Get the centroid of the polygon
centroid = gnut.geometry().centroid().getInfo()
print(f"Centroid: {centroid}")

# Get the bounding box as a GeoJSON
bounding_box = gnut.geometry().bounds().getInfo()
print(f"Bounding Box: {bounding_box}")



### Estimate the each polygon area (m2), for each crop e.g Groundnt


In [ ]:
# Define a function to calculate the area of each polygon in hectares
#def calc_area(feature):
    #return feature.set('area_ha', ee.Number(feature.geometry().area().divide(10000)).format('%.3f'))

# Define a function to calculate the area of each polygon in m2
def calc_area(feature):
  return feature.set('area_m2', ee.Number(feature.geometry().area())
  .format('%.1f'))#.divide(10000)) & # .format('%.1f') use to limit the digits

# Apply the function to each feature in the collection
area_mapped = merged2023.map(calc_area)

# Fetch the mapped area information from the server
areas_info = area_mapped.limit(10).getInfo()  # Limiting to the first 10 features directly to show everthing remove .limit(10)

# Iterate through the first 10 features and print area in hectares with 3 decimal places
for feature in areas_info['features']:
    area = feature['properties']['area_m2'] ##m2
    print(f"Gnut Polygon_Area: {area} m2") #m2


### Area (ha) distributions e.g Groundnut

In [ ]:
# Define a function to calculate the area of each polygon in hectares
#def calc_area(feature):
    #return feature.set('area_ha', feature.geometry().area().divide(10000))  # Convert from square meters to hectares

# Define a function to calculate the area of each polygon in hectares
def calc_area(feature):
    return feature.set('area_ha', feature.geometry().area().divide(10000)) #--> # Convert from square meters to hectares

# Apply the function to each feature in the collection
area_mapped = gnut.map(calc_area)

# Extract and print the area of each polygon
areas_info = area_mapped.getInfo()

# Extracting areas into a list for plotting
areas = [feature['properties']['area_ha'] for feature in areas_info['features']]

# Plotting the distribution of polygon areas
plt.figure(figsize=(10, 6))
plt.hist(areas, bins='auto', color='skyblue', alpha=0.8, rwidth=0.85)
plt.title('Distribution of Polygon Areas in Ha for Groundnut')
plt.xlabel('Area (Ha)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

Show area of each crop

In [ ]:
#  'gnut', 'gsorrel', 'rice', etc. are already defined as ee.Geometry() or ee.Feature() objects representing each crop
crop_types = {
    'Gnut': gnut,
    'G. Sorrel': gsorrel,
    'Rice': rice,
    'Sesame': sesame,
    'Soye': soye,
    'Watermelon': melon,
    'Cassava': cassava,
    'Gnut Mix': gnutmix,
    'Millet Mix': milletmix,
    'Okra': okra,
    'Sorgh': sorgh,
    'Wheat': wheat,
    'Millet': millet,
    'Cowpea Mix': cowpmix,
    'Fallow': fallow
}

# Calculate area for each crop and store in a dictionary
#areas = {name: crop.geometry().area().divide(10000).getInfo() for name, crop in crop_types.items()} #In Hectares
areas = {name: crop.geometry().area().getInfo() for name, crop in crop_types.items()}


# Create a DataFrame from the area dictionary
area_df = pd.DataFrame(list(areas.items()), columns=['Crop Type', 'Area (Squere meteres M2)'])

# Display the DataFrame
print(area_df)

area_df.head(14)

Graphic of each crop in area coverages

In [ ]:
area_df = pd.DataFrame(list(areas.items()), columns=['Crop Type', 'Area (M2)'])

# Sort the DataFrame by area for better visualization
area_df.sort_values('Area (M2)', ascending=False, inplace=True)

# Plotting
plt.figure(figsize=(12, 8))
plt.bar(area_df['Crop Type'], area_df['Area (M2)'], color='red')
plt.xlabel('Crop Type')
plt.ylabel('Area in M2')
plt.title('Area of Each Crop Type for FY2023')
plt.xticks(rotation=90)  # Rotate crop type names for better readability
plt.grid(False)
plt.show()

### Count Pixels (NICFI) for each crop field represnetation of each specteral bands e.g Groundnut

In [ ]:
# time range
start_date = '2023-09-01'
end_date = '2023-09-30'

# Load the Planet/NICFI imagery
imagery = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa').filterDate(start_date, end_date).filterBounds(merged2023)  # Assuming 'gnut' is the ee.Geometry() of my area of interest
#bands info
bands = ee.ImageCollection(imagery).first()
print(bands.bandNames().getInfo())

#################  Planet Pixel count ####### Actual pixel count from PLANET

# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Mask the image with the polygon
    masked_image = image.clip(gnut) #remember 1 polygone selected for closer see demo

    # Count pixels - 4.77 X 4.77 m (~5X5m) actual Planet/NICFI pixel resolution
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut,
        scale=4.77,  # Scale in meters; 5m resolution take time to estimate (timeout)
        # to avoid time out estimation use 10X10m and convert it in to 5X5 by devide the result 4
        maxPixels=1e9  # Adjust maxPixels if needed to handle large areas
    )

    # Need to return an ee.Feature for the .map() function
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])



In [ ]:
# Define the time range
start_date = '2023-09-01'
end_date = '2023-09-30'


# Load the Planet/NICFI imagery
imagery = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
            .filterDate(start_date, end_date) \
            .filterBounds(merged2023)

# Print band information
bands = imagery.first()
print(bands.bandNames().getInfo())

# Function to mask and count pixels within the AOI
def count_pixels(image):
    # Mask the image with the polygon
    masked_image = image.clip(merged2023)

    # Count pixels - 10m resolution for faster computation
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=merged2023,
        scale=10,  # Scale in meters; coarser resolution for faster computation
        maxPixels=1e6  # Adjust maxPixels if needed to handle large areas
    )

    # Need to return an ee.Feature for the .map() function
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

### pixel esimates for each crop considering area not pixels

In [ ]:
# Assuming these are the individual crop polygons. others to be added from other crop years.
crop_types = {
    'gnut': gnut, 'gsorrel': gsorrel, 'rice': rice, 'sesame': sesame,
    'soye': soye, 'watermelon': melon, 'cassava': cassava,
    'gnutmix': gnutmix, 'milletmix': milletmix, 'okra': okra,
    'sorgh': sorgh, 'wheat': wheat, 'millet': millet, 'cowpmix': cowpmix,
    'fallow': fallow
}

def count_pixels(crop_name, crop_polygon):
    # Filter the imagery to the bounds of the crop polygon
    crop_imagery = imagery.filterBounds(crop_polygon)

    # Apply the pixel counting for each image in the filtered collection
    def pixel_count(image):
        masked_image = image.clip(crop_polygon)
        pixel_count = masked_image.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=crop_polygon,
            scale=10,  # Adjust the scale based on the actual resolution of NICFI imagery
            maxPixels=1e9
        )
        return ee.Feature(None, pixel_count)

    pixel_counts = crop_imagery.map(pixel_count)
    pixel_counts_info = pixel_counts.getInfo()

    # Print results for each crop type
    print(crop_name)
    for feature in pixel_counts_info['features']:
        print(feature['properties'])

# Run the pixel counting for each crop type
for crop_name, crop_polygon in crop_types.items():
    count_pixels(crop_name, crop_polygon)

### Pixel Exclusion representes less (e.g 40%) with in inside the polygon area.

Masking Pixels: Pixels with less than 40% coverage by the polygon are excluded using the mask method, where mask = fraction.gte(0.4) creates a mask that includes only pixels where the fraction is 40% or more.

- Band Selection: When creating the mask, the code now ensures that a single band is selected using image.select(0). This selects the first band of the image, assuming the first band is suitable for your masking purposes. Adjust the band selection if necessary based on the bands available in your specific imagery.

- Mask Application: By using single_band_image.mask() in the fraction calculation, we ensure that any operations involving masks deal with a single-band image, thus avoiding band mismatch errors.

In [ ]:
# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Ensure that we work with a single band for mask creation
    # Here, you could potentially select a specific band or reduce to a single band
    # As an example, if the image has multiple bands, use just one (e.g., the first band) for mask calculations
    single_band_image = image.select(1)

    # Calculate the fraction of the pixel area that overlaps with the polygon using a constant image
    constantImage = ee.Image.constant(1).clip(okra)
    fraction = constantImage.updateMask(single_band_image.mask()).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=gnut,
        scale=5,  # estimation with planet 5x5 is very difficult
        maxPixels=1e9
    ).get('constant')

    # Threshold the fraction to include only pixels with >= 40% coverage
    mask = ee.Image(fraction).gte(0.4)

    # Mask the image with the calculated mask
    masked_image = image.updateMask(mask)

    # Count pixels
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut,
        scale=5,
        maxPixels=1e9
    )

    # Return an ee.Feature for the .map() function to work properly
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

only one polygon from gnut only for learning use

In [ ]:
# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Ensure that we work with a single band for mask creation
    single_band_image = image.select(1)  # Select the first band (assuming 1-based index)

    # Calculate the fraction of the pixel area that overlaps with the polygon using a constant image
    constant_image = ee.Image.constant(1).clip(polygon1)
    fraction = constant_image.updateMask(single_band_image.mask()).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=gnut.geometry(),
        scale=5,  # estimation with planet 5x5 is very difficult
        maxPixels=1e9
    ).get('constant')

    # Threshold the fraction to include only pixels with >= 40% coverage
    mask = ee.Image.constant(fraction).gte(0.4)

    # Mask the image with the calculated mask
    masked_image = image.updateMask(mask)

    # Count pixels
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut.geometry(),
        scale=5,
        maxPixels=1e9
    )

    # Return an ee.Feature for the .map() function to work properly
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

In [ ]:
# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Create a single band composite by averaging RGB and NIR bands
    composite = image.expression(
        "(R + G + B + N) / 4",  # Expression combining the bands
        {
            'R': image.select('R'),  #  'R' is the Red band
            'G': image.select('G'),  # 'G' is the Green band
            'B': image.select('B'),  #  'B' is the Blue band
            'N': image.select('N')   #  'N' is the NIR band
        }
    )

    # Generate a mask based on the composite value, adjusting threshold as necessary
    mask = composite.gt(0.2)  # Example threshold

    # Mask the image with the generated mask
    masked_image = image.updateMask(mask)

    # Count pixels
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut,
        scale=5,
        maxPixels=1e9
    )

    # Return an ee.Feature for the .map() function to work properly
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

Pixel counts using area coverage regardless of the specteral bands (NICFI + 4.77m X 4.77 M resolutions)

In [ ]:
resolution = 4.77  # Resolution of NICFI imagery in meters (4.77m x 4.77m per pixel)


# Function to calculate pixel count
def calculate_pixels(crop):
    area = crop.geometry().area()  # Area in square meters
    pixel_area = resolution * resolution  # Area of one pixel in square meters
    pixel_count = area.divide(pixel_area)  # Number of pixels
    return pixel_count.getInfo()

# Calculate pixel count for each crop and store in a dictionary
pixel_counts = {name: calculate_pixels(crop) for name, crop in crop_types.items()}

# Create a DataFrame from the area and pixel count dictionaries
area_df = pd.DataFrame(list(pixel_counts.items()), columns=['Crop Type', 'Pixel Count'])

# Sort the DataFrame by 'Pixel Count' in descending order
area_df = area_df.sort_values(by='Pixel Count', ascending=False)
# Display the DataFrame
print(area_df)

In [ ]:
# Plotting
#plt.figure(figsize=(10, 6))
#plt.bar(area_df['Crop Type'], area_df['Pixel Count'], color='green')
#plt.xlabel('Crop Type')
#plt.ylabel('Pixel Count')
#plt.title('Pixel Count by Crop Type (Ascending Order)')
#plt.xticks(rotation=45, ha='right')
#plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
#plt.show()
####################
# Plotting using seaborn lib.
plt.figure(figsize=(12, 8))
sns.barplot(x='Crop Type', y='Pixel Count', data=area_df, palette='viridis')  # Using 'viridis' palette for varying colors
plt.xlabel('Crop Type')
plt.ylabel('Pixel Count')
plt.title('Pixel Count by Crop Type - for data base 2(FY2023)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()

Area vs Pixel count

In [ ]:
# Assuming 'crop_types' dictionary is predefined with ee.Geometry() objects
resolution = 4.77  # Resolution of NICFI imagery in meters

# Function to calculate pixel count
def calculate_pixels(area):
    pixel_area = resolution * resolution  # Area of one pixel in square meters
    return area / pixel_area

# Calculate area and pixel count for each crop and store in dictionaries
area_and_pixels = []
for name, crop in crop_types.items():
    area = crop.geometry().area().getInfo()  # Area in square meters
    pixel_count = calculate_pixels(area)
    area_and_pixels.append((name, area, pixel_count))

# Create a DataFrame from the results
df = pd.DataFrame(area_and_pixels, columns=['Crop Type', 'Area (Square Meters)', 'Pixel Count'])

# Sort the DataFrame by 'Pixel Count' in descending order (optional)
df.sort_values(by='Pixel Count', ascending=False, inplace=True)

# Plotting
fig, ax1 = plt.subplots(figsize=(14, 8))

# Bar plot for Area using Seaborn
color = 'tab:red'
sns.barplot(x='Crop Type', y='Area (Square Meters)', data=df, palette='viridis', ax=ax1)
ax1.set_xlabel('Crop Type')
ax1.set_ylabel('Area (Square Meters)', color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a twin Axes sharing the x-axis for Pixel Count
ax2 = ax1.twinx()
color = 'tab:blue'
sns.lineplot(x='Crop Type', y='Pixel Count', data=df, sort=False, marker='o', color='red', ax=ax2)
ax2.set_ylabel('Pixel Count', color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Improve layout and set x-axis labels rotation
plt.xticks(rotation=90)
fig.tight_layout()

plt.show()

### Garba

Hi Garba, please use the above functions and generate the mean/median pixel specteral band values for "R", "G", "B" and "N" values of each pixel for each crop. Let me know if you need my assitance or have questions on the above functions. Perhaps you can show it in vector/tabular form.

--------------------------------
R      |    G  |     B  |     N
-------------------------------
       |       |        |  

**Steps**

To enhance the quality and usability of the images for further analysis or applications Normalization and calibration of images are essential processes in image processing and computer vision.  

**1. Data Collection**: Obtain multispectral images of the crops that include the required spectral bands (R, G, B, and NIR).

**2. Image Preprocessing**:

2.1. Image Normalization

In [ ]:


# Load the image
image = cv2.imread('/path/to/your/image.jpg')

# Convert BGR image to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Normalize the image to range [0, 1]
normalized_image = image / 255.0

# Alternatively, you can normalize to range [-1, 1]
# normalized_image = (image / 127.5) - 1.0

# Display the original and normalized images
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(image)

plt.subplot(1, 2, 2)
plt.title('Normalized Image')
plt.imshow(normalized_image)

plt.show()


2.2. Calibration

In [ ]:
import cv2
import numpy as np
import glob

# Termination criteria for corner sub-pixel accuracy
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0)
objp = np.zeros((6*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

# Arrays to store object points and image points from all images
objpoints = []  # 3d points in real world space
imgpoints = []  # 2d points in image plane

# Load calibration images
images = glob.glob('/path/to/calibration/images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)

    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7, 6), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Perform camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Save the calibration results
np.savez('calibration_data.npz', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)

# Undistort an image using the calibration results
img = cv2.imread('/path/to/your/test/image.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# Undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# Crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

# Display the result
cv2.imshow('calibrated', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()


2.3. Alignment

  **  Install OpenCV**: Install OpenCV in your Colab environment to use its functionalities.

    **Load and Display Images**: Load the two images you want to align. Display them using Matplotlib to ensure they are loaded correctly.

    **Detect ORB Features and Compute Descriptors**: Use the ORB detector to find keypoints and compute descriptors for both images.

    **Match Features Using BFMatcher**: Match the descriptors using BFMatcher, and sort the matches based on their distance (quality).

    **Find Homography and Warp Image**: Extract the coordinates of the matched points, compute the homography matrix using RANSAC, and apply the homography to warp the second image to align it with the first image.

1. **Install OpenCV**: First, ensure you have OpenCV installed in your Colab environment.

In [ ]:
!pip install opencv-python-headless


**2. Load and Display Images**: Load the images you want to align.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the images in grayscale
img1 = cv2.imread('/path/to/your/image1.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/path/to/your/image2.jpg', cv2.IMREAD_GRAYSCALE)

# Display the images
plt.subplot(1, 2, 1)
plt.title('Image 1')
plt.imshow(img1, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Image 2')
plt.imshow(img2, cmap='gray')

plt.show()


**3. Detect ORB Features and Compute Descriptor**s:

In [ ]:
# Initialize the ORB detector
orb = cv2.ORB_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints2, descriptors2 = orb.detectAndCompute(img2, None)


**4. Match Features Using BFMatcher**:

In [ ]:
# Create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors
matches = bf.match(descriptors1, descriptors2)

# Sort them in the order of their distance
matches = sorted(matches, key=lambda x: x.distance)

# Draw top matches
img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matches
plt.figure(figsize=(20, 10))
plt.imshow(img_matches)
plt.title('Feature Matches')
plt.show()


**5. Find Homography and Warp Image:**

In [ ]:
# Extract location of good matches
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

# Find homography matrix
h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)

# Use homography to warp the image
height, width = img1.shape
aligned_img = cv2.warpPerspective(img2, h, (width, height))

# Display the aligned image
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.title('Reference Image')
plt.imshow(img1, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Aligned Image')
plt.imshow(aligned_img, cmap='gray')

plt.show()


**Extract Pixel Values: Extract the pixel values for each spectral band (R, G, B, NIR) for each segmented crop area.**

Clipping

3. **Segmentation**

**4. Extract Pixel Values**: Extract the pixel values for each spectral band (R, G, B, NIR) for each segmented crop area.

**5. Compute Statistics:**

    Mean: Calculate the mean value for each spectral band across all pixels for each crop.
    Median: Calculate the median value for each spectral band across all pixels for each crop.

### Cyrille


**GARBA**  Other Proposition

[**Click Here for the first step to follow**](https://code.earthengine.google.com/91904df41cc77089f346977c173a0122)

Change the variable to our variable

1.   Area of Interest(roi)
2.   Land cover entities
3.   Filter the date
4.   Setup the other parameters for the convenance  


Install some packages


In [ ]:
!pip install rasterio
!pip install earthpy



In [ ]:
#Import packages
import pandas as pd
import numpy as np
import keras
from keras import sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Input, GlobalMawPooling1D
from keras.callbacks import EarlyStopping
from kera import Model
import rasterio
import earth.plot as ep
from keras.utils import to_categorical
import sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, Classification_report
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors

In [ ]:
#Mount GDRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at/content/drive; to attemp to forcibly remount, call drive.mount("/content/drive" force_remount=True)

In [ ]:
# Parameter
FEATURES = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'EVI', 'NBR', 'NDMI', 'NDWI', 'NDBI', 'NDBaI', 'elevation']
LABEL = ['classvalue']
SPLIT = ['sample']
N_CLASSES = 9
CLASSES = [1, 2, 3, 4, 5, 6, 7, 8, 9]
PALETTE = ['#F08080', '#D2B48C', '#87CEFA', '#008080', '#90EE90', '#228B22', '#808000', '#FF8C00', '#006400']
SAMPLE_PATH = '/content/drive/MyDrive/DL/Samples_LC_Jambi_2023.csv'
IMAGE_PATH = '/content/drive/MyDrive/DL/Landsat_Jambi_2023.tif'

In [ ]:
# Load image
image = rasterio.open(IMAGE_PATH)
bandNum = image.count
height = image.height
width = image.width
crs = image.crs
transform = image.transform
shape = (height, width)

image_vis = []
for x in [6, 5, 4]:
  image_vis.append(image.read(x))
image_vis = np.stack(image_vis)

plot_size = (8, 8)
ep.plot_rgb(
  image_vis,
  figsize=plot_size,
  stretch=True,
)

In [ ]:
# Read sample
samples = pd.read_csv(SAMPLE_PATH)
samples = samples.sample(frac = 1) # Shuffle data
samples

In [ ]:
# Split into train and test based on column
train = samples[samples['sample'] == 'train']
test = samples[samples['sample'] == 'test']

# Split between features and label
train_features = train[FEATURES]
train_label = train[LABEL]
test_features = test[FEATURES]
test_label = test[LABEL]

# Function to reshape array input
def reshape_input(array):
  shape = array.shape
  return array.reshape(shape[0], shape[1], 1)

# Convert samples dataframe (pandas) to numpy array
train_input = reshape_input(train_features.to_numpy())
test_input = reshape_input(test_features.to_numpy())

# Also make label data to categorical
train_output = to_categorical(train_label.to_numpy(), N_CLASSES + 1, int)
test_output = to_categorical(test_label.to_numpy(), N_CLASSES + 1, int)

# Show the data shape
print(f'Train features: {train_input.shape}\nTest features: {test_input.shape}\nTrain label: {train_output.shape}\nTest label: {test_output.shape}')

Train features: (14853, 14, 1)
Test features: (4043, 14, 1)
Train label: (14853, 10)
Test label: (4043, 10)

In [ ]:
# Make model for our data
# Input shape
train_shape = train_input.shape
input_shape = (train_shape[1], train_shape[2])

# Model parameter
neuron = 64
drop = 0.2
kernel = 2
pool = 2

# Make sequential model
model = Sequential([
  Input(input_shape),
  Conv1D(neuron * 1, kernel, activation='relu'),
  Conv1D(neuron * 1, kernel, activation='relu'),
  MaxPooling1D(pool),
  Dropout(drop),
  Conv1D(neuron * 2, kernel, activation='relu'),
  Conv1D(neuron * 2, kernel, activation='relu'),
  MaxPooling1D(pool),
  Dropout(drop),
  GlobalMaxPooling1D(),
  Dense(neuron * 2, activation='relu'),
  Dropout(drop),
  Dense(neuron * 1, activation='relu'),
  Dropout(drop),
  Dense(N_CLASSES + 1, activation='softmax')
])

model.summary()

In [ ]:
# Train the model

# Compline the model
model.compile(
    optimizer='Adam',
    loss='CategoricalCrossentropy',
    metrics=['accuracy']
)

# Create callback to stop training if loss not decreasing
stop = EarlyStopping(
    monitor='loss',
    patience=5
)

# Fit the model
result = model.fit(
    x=train_input, y=train_output,
    validation_data=(test_input, test_output),
    batch_size=1024,
    callbacks=[stop],
    epochs=100,
)

In [ ]:
# Show history
history = pd.DataFrame(result.history)

plt.figure(figsize = (10, 8))
plt.plot(range(len(history['accuracy'].values.tolist())), history['accuracy'].values.tolist(), label = 'Train_Accuracy')
plt.plot(range(len(history['loss'].values.tolist())), history['loss'].values.tolist(), label = 'Train_Loss')
plt.plot(range(len(history['val_accuracy'].values.tolist())), history['val_accuracy'].values.tolist(), label = 'Test_Accuracy')
plt.plot(range(len(history['val_loss'].values.tolist())), history['val_loss'].values.tolist(), label = 'Test_Loss')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
# Predict test data
prediction = np.argmax(model.predict(test_input), 1).flatten()
label = np.argmax(test_output, 1).flatten()

# Confusion matrix
cm = confusion_matrix(label, prediction, normalize='true')
cm = ConfusionMatrixDisplay(cm)
cm.plot()

# Classification report
print(classification_report(label, prediction))

In [ ]:
# Predict image using the model
image_input = []
for x in range(14):
  image_input.append(image.read(x + 1))
image_input = reshape_input(np.stack(image_input).reshape(14, -1).T)

# Predict
prediction = model.predict(image_input, batch_size=4096*20)
prediction = np.argmax(prediction, 1)
prediction = prediction.reshape(shape[0], shape[1])

# Visualize
cmap, norm = from_levels_and_colors(CLASSES, PALETTE, extend='max')
ep.plot_bands(prediction, cmap=cmap, norm=norm, figsize=plot_size)

In [ ]:
# Save file to drive
save_location = '/content/drive/MyDrive/DL/'
name = 'LC_Jambi_2023.tif'
location = save_location + name

new_dataset = rasterio.open(
      location,
      mode='w', driver='GTiff',
      height = prediction.shape[0], width = prediction.shape[1],
      count=1, dtype=str(prediction.dtype),
      crs=crs,
      transform=transform
)
new_dataset.write(prediction, 1);
new_dataset.close()

For following step by step [Youtube Chanel](https://www.youtube.com/watch?v=NFoZPyQqVRA) **source**: Ramadhan